# Preparation

## Load bibs

In [ ]:
import gc # Garbage Collector
import os
import pickle
import random
import re
import textwrap
from collections import Counter
from io import StringIO

import hdbscan
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display # Import display for better DataFrame rendering
from matplotlib.patches import Patch
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap


from scipy.stats import spearmanr
from shapely.geometry import MultiPolygon, Polygon
from sklearn import metrics
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.manifold import TSNE
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    jaccard_score,
    silhouette_score,
    silhouette_samples,
)
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tqdm.auto import tqdm # For progress bars (preferred over from tqdm import tqdm)

import geopandas as gpd
from umap import UMAP

## Load data

### Shape files

In [ ]:
def filter_alaska_polygons(gdf):
    """
    Goal:
        Remove remote parts of Alaska from the geometry data, 
        particularly those located far to the west (e.g., small island chains).
        
    Approach:
        - Identify the geometry for Alaska.
        - If it's a MultiPolygon, filter out parts whose first coordinate has x > 150.
        - If it's a single Polygon and its x > 150, discard it.
        - Update the geometry or remove Alaska completely if no valid parts remain.
    
    Output:
        Returns the modified GeoDataFrame with Alaska filtered accordingly.
    """
    # Find the row index for Alaska
    alaska_idx = gdf[gdf['NAME'] == 'Alaska'].index

    # If Alaska is not present, return the GeoDataFrame unchanged
    if alaska_idx.empty:
        return gdf

    # Extract Alaska's geometry
    alaska_geometry = gdf.loc[alaska_idx[0], 'geometry']
    updated_geometry = None

    if isinstance(alaska_geometry, MultiPolygon):
        # Keep only those polygons whose first x-coordinate is <= 150
        filtered_polys = [
            poly for poly in alaska_geometry.geoms 
            if poly.exterior.coords[0][0] <= 150
        ]
        if filtered_polys:
            updated_geometry = MultiPolygon(filtered_polys)
    elif isinstance(alaska_geometry, Polygon):
        # Keep the polygon only if its first x-coordinate is <= 150
        if alaska_geometry.exterior.coords[0][0] <= 150:
            updated_geometry = alaska_geometry

    if updated_geometry is not None:
        # Update the geometry in the GeoDataFrame
        gdf.loc[alaska_idx, 'geometry'] = updated_geometry
    else:
        # Remove Alaska from the GeoDataFrame if no valid geometry remains
        gdf = gdf.drop(index=alaska_idx)

    return gdf


In [ ]:
# --- Goal ---
# Prepare U.S. state and county geometries for mapping and analysis,
# with a specific focus on filtering out distant areas (e.g. Hawaii, Puerto Rico, etc.)
# and modifying Alaska to exclude remote island groups.

# --- Approach ---
# - Load state and county shapefiles from pickle files
# - Remove non-continental U.S. territories and Hawaii
# - Separate Alaska and the continental U.S. into different GeoDataFrames
# - Remove an outlier county (FIPS '02016') from both Alaska and continental counties

# --- Output ---
# Cleaned GeoDataFrames for:
# - state and county shapes for continental U.S.
# - state and county shapes for Alaska

import pandas as pd

# Load original shapefiles
state_shape = pd.read_pickle('data/original_data/pkl/state.pickle')
county_shape = pd.read_pickle('data/original_data/pkl/county.pickle')

# Filter Alaska polygons to remove remote parts
state_shape = filter_alaska_polygons(state_shape)

# Rename county GEOID column to match expected FIPS naming
county_shape = county_shape.rename(columns={'GEOID': 'FIPS'})

# Filter STATEFP codes for continental U.S. (excluding non-continental territories and Hawaii)
filtered_statefp_north_america = state_shape.loc[
    ~state_shape['NAME'].isin([
        'Hawaii', 
        'Puerto Rico', 
        'Commonwealth of the Northern Mariana Islands', 
        'American Samoa', 
        'United States Virgin Islands', 
        'Guam'
    ]), 
    'STATEFP'
]

# Filter STATEFP for Alaska only
filtered_statefp_alaska = state_shape.loc[
    state_shape['NAME'] == 'Alaska', 
    'STATEFP'
]

# Create separate GeoDataFrames for continental U.S. and Alaska (states)
filtered_state_shape_north_america = state_shape[state_shape['STATEFP'].isin(filtered_statefp_north_america)]
filtered_state_shape_alaska = state_shape[state_shape['STATEFP'].isin(filtered_statefp_alaska)]

# Create separate GeoDataFrames for continental U.S. and Alaska (counties)
# Also remove the outlier county with FIPS '02016' (St. Paul Island, Alaska)
filtered_county_shape_north_america = county_shape[
    county_shape['STATEFP'].isin(filtered_statefp_north_america) & 
    (county_shape['FIPS'] != '02016')
]
filtered_county_shape_alaska = county_shape[
    county_shape['STATEFP'].isin(filtered_statefp_alaska) & 
    (county_shape['FIPS'] != '02016')
]


### County data

In [ ]:
county_master = pd.read_pickle('data/original_data/pkl/county_information.pkl')
county_master.head()

## Visualization of US industry regions

In [ ]:

# --- Goal ---
# Assign U.S. states to broader industry regions based on their GEOID,
# and identify any states that are associated with more than one region.

# --- Approach ---
# - Read region classification data from a string into a DataFrame
# - Merge this data with the cleaned state geometry DataFrame (filtered_state_shape_north_america)
# - Fill in missing regions with 'Other'
# - Group by state name and count how many distinct regions each state appears in

# --- Output ---
# List of states that are assigned to more than one region (if any)

# Define region classification data as CSV string
data = """
State ID,GEOID,State,Region
IL,17,Illinois,North-Eastern
KS,20,Kansas,Southern
IN,18,Indiana,North-Eastern
MI,26,Michigan,North-Eastern
OH,39,Ohio,North-Eastern
WI,55,Wisconsin,North-Eastern
CT,09,Connecticut,New England
ME,23,Maine,The New England
MA,25,Massachusetts,New England
NH,33,New Hampshire,New England
RI,44,Rhode Island,New England
VT,50,Vermont,New England
DE,10,Delaware,New York and Mid-Atlantic
DC,11,District of Columbia,New York and Mid-Atlantic
MD,24,Maryland,New York and Mid-Atlantic
NJ,34,New Jersey,New York and Mid-Atlantic
NY,36,New York,New York and Mid-Atlantic
PA,42,Pennsylvania,New York and Mid-Atlantic
CA,06,California,Pacific Coastal
OR,41,Oregon,Pacific Coastal
WA,53,Washington,Pacific Coastal
AL,01,Alabama,Southern
AR,05,Arkansas,Southern
FL,12,Florida,Southern
GA,13,Georgia,Southern
LA,22,Louisiana,Southern
MS,28,Mississippi,Southern
OK,40,Oklahoma,Southern
TN,47,Tennessee,Southern
TX,48,Texas,Southern
AZ,04,Arizona,Western
CO,08,Colorado,Western
NV,32,Nevada,Western
NM,35,New Mexico,Western
UT,49,Utah,Western
WY,56,Wyoming,Western
PR,72,Puerto Rico,Other
AS,60,American Samoa,Other
AK,02,Alaska,Other
HI,15,Hawaii,Other
GU,66,Guam,Other
VI,78,U.S. Virgin Islands,Other
MP,69,Northern Mariana Islands,Other
"""

# Read the region data into a DataFrame
region_df = pd.read_csv(StringIO(data), dtype={'GEOID': str})

# Merge with filtered state geometries to assign regions
industry_regions = filtered_state_shape_north_america.merge(region_df, on="GEOID", how="left")

# Fill any missing regions with 'Other'
industry_regions['Region'] = industry_regions['Region'].fillna('Other')

# Identify states that appear in more than one region
multi_region_states = (
    industry_regions.groupby("State")["Region"]
    .nunique()
    .loc[lambda x: x > 1]
    .index
    .tolist()
)

# Output the result
print("States assigned to multiple regions:", multi_region_states)


In [ ]:
# --- Goal ---
# Identify U.S. territories or states from the master dataset that are 
# not included in the industry_regions DataFrame (e.g., excluded areas like Puerto Rico or Guam).

# --- Approach ---
# - Perform a left merge between the county master and the industry regions on state FIPS code
# - Use the merge indicator to find unmatched entries (i.e., those only in the county master)
# - Extract unique state FIPS codes and state names for the missing entries

# --- Output ---
# List of U.S. territories or states (statefp and state_name) not found in the industry region data

# Perform a left join to compare presence in both datasets
schnitttmenge = pd.merge(
    county_master[['statefp', 'state_name']],  # Include state_name for context
    industry_regions[['STATEFP']],
    left_on='statefp',
    right_on='STATEFP',
    how='left',
    indicator=True  # Adds column '_merge' to indicate match status
)

# Filter for rows that exist only in the county_master (i.e., no match in industry_regions)
fehlende_werte = schnitttmenge[schnitttmenge['_merge'] == 'left_only']

# Keep only unique statefp and state_name entries
unique_fehlende_werte = fehlende_werte[['statefp', 'state_name']].drop_duplicates()

# Create a list of the missing statefp values
us_territories = unique_fehlende_werte['statefp'].to_list()

# Output the missing states/territories
print(unique_fehlende_werte)


In [ ]:
# --- Goal ---
# Remove FIPS entries from U.S. territories not included in the analysis.
# Compare the number of unique FIPS codes before and after filtering.

df_original = pd.read_pickle("data/processed_data/pkl/feature_df.pkl")
print(df_original['FIPS'].nunique())

df_original = df_original[~df_original['FIPS'].str[:2].isin(us_territories)].reset_index(drop=True)
print(df_original['FIPS'].nunique())


In [ ]:
df_original

# Data Preparation 

## Scaling

In [ ]:

# --- Data Preprocessing for Log10 Scaling ---
# Applies Log10 scaling to features, safely handling zeros and NaNs by replacing them
# with a small positive value to prevent mathematical errors.

# Define columns to be excluded from scaling (e.g., identifiers)
columns_to_exclude_from_scaling = ['FIPS']

# Extract ID columns
df_ids = df_original[columns_to_exclude_from_scaling].copy()

# Isolate raw feature columns for scaling
df_raw_features = df_original.drop(columns=columns_to_exclude_from_scaling)

# Define a small positive value to replace zeros and NaNs before log transformation
epsilon_value = 0.001

# Replace zeros and NaNs in features with epsilon_value
df_features_for_log = df_raw_features.replace(0, epsilon_value).fillna(epsilon_value)

# Perform Log10 transformation
df_log_transformed_features = np.log10(df_features_for_log)

# Ensure transformed data retains original column names and index
df_log_transformed_features = pd.DataFrame(
    df_log_transformed_features,
    columns=df_features_for_log.columns,
    index=df_features_for_log.index
)

# Concatenate ID columns with log-transformed features
df_scaled_log = pd.concat([df_ids, df_log_transformed_features], axis=1)

print("\nData preprocessing complete. Preview of df_scaled_log:")
display(df_scaled_log.head()) # Use display() for better rendering

# Optional: Access to the original (non-scaled) feature values for reference
df_original_features = df_original.drop(columns=columns_to_exclude_from_scaling)

In [ ]:
feature_id_dict = {feature: "F" + str(i+1) for i, feature in enumerate(df_original_features.columns)}
feature_id_dict

In [ ]:
# --- Goal ---
# Compare the distribution of feature values before and after log10 scaling using scatter plots.
# Visualize each feature as a vertical column of points, labeled with its assigned feature ID.

columns_original = df_original.columns[1:]  # Skip first column (e.g., FIPS)
df_features = df_original[columns_original]

# Assign feature IDs like F1, F2, ... for better readability in plots
feature_id_dict = {feature: "F" + str(i + 1) for i, feature in enumerate(columns_original)}
feature_ids = [feature_id_dict[col] for col in columns_original]

# Plot setup
ss = 10  # Marker size
fig, axs = plt.subplots(1, 2, figsize=(9, 3), sharey=False)

# Scatter plot: Original features
for i, col in enumerate(columns_original):
    axs[0].scatter(np.full(len(df_original[col]), i), df_original[col], alpha=0.5, s=ss)
axs[0].set_title('Original Features')
axs[0].set_xticks(range(len(columns_original)))
axs[0].set_xticklabels(feature_ids, rotation=90)
axs[0].set_ylabel('Value')
for i, col in enumerate(columns_original):
    axs[0].set_ylim(df_original[col].min() - 2000, df_original[col].max() + 2000)

# Scatter plot: Log10-scaled features
for i, col in enumerate(columns_original):
    axs[1].scatter(np.full(len(df_scaled_log[col]), i), df_scaled_log[col], alpha=0.5, s=ss)
axs[1].set_title('Log10 Scaled Features')
axs[1].set_xticks(range(len(columns_original)))
axs[1].set_xticklabels(feature_ids, rotation=90)
for i, col in enumerate(columns_original):
    axs[1].set_ylim(df_scaled_log[col].min() - 1., df_scaled_log[col].max() + 2)


plt.tight_layout()
plt.show()


In [ ]:
def plot_histograms(df_original, df_scaled_log):
    # Get feature columns (excluding the first, e.g., ID column)
    columns_original = df_original.columns[1:]
    
    # Define scaling datasets and their labels
    datasets = [
        (df_scaled_log, 'Log10 Scaled')
    ]
    
    # Create one row of subplots per feature
    for col in columns_original:
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))
        
        # Histogram of original values
        axs[0].hist(df_original[col], bins=30, alpha=0.7, edgecolor='black')
        axs[0].set_title(f'{col} - Original')
        axs[0].set_xlabel('Value')
        axs[0].set_ylabel('Frequency')
        
        # Histogram of log10-scaled values
        axs[1].hist(df_scaled_log[col], bins=30, alpha=0.7, edgecolor='black')
        axs[1].set_title(f'{col} - Log10 Scaled')
        axs[1].set_xlabel('Value')
        axs[1].set_ylabel('Frequency')

              
        # Title for the full figure
        fig.suptitle(f'Histograms for {col}', fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()
plot_histograms(df_original, df_scaled_log)

In [ ]:
df_scaled = df_scaled_log
df_scaled_features = df_scaled_log.iloc[:, 1:]

In [ ]:
# --- Goal ---
# Create a structured summary table for all features, including metadata, statistics, and data types.
# Ensure consistent formatting for display and replace missing values where necessary.

# Build summary DataFrame without rounding numerical stats
summary = pd.DataFrame({
    'Feature': df_original_features.columns,
    'ID': ["F" + str(i + 1) for i in range(len(df_original_features.columns))],
    'Description': [
        "SOC 51-9022, Grinding, Polishing by Hand",
        "SOC 51-4121, Welders, Cutters, Solderers",
        "SOC 49-9041, Industrial Machinery Mechanics",
        "SOC 49-9071, Maintenance and Repair Workers",
        "SOC 51-4033, Grinding, Lapping, Polishing",
        "SOC 51-4035, Milling and Planing Machine Setters",
        "SOC 47-2211, Sheet Metal Workers",
        "SOC 51-2041, Structural Metal Fabricators",
        "NAICS 3315, Foundries", 
        "NAICS 3364, Aerospace",
        "NAICS 3366, Shipbuilding",
        "NAICS 3335, Metalworking Machines Manufacturing", 
        "NAICS 3320A1, Steel forming", 
        "NAICS 3320A2, Structural Metals Manufacturing",
        "NAICS 3327, Machine Shops", 
        "NAICS 3312, Steel Product Manufacturing", 
        "NAICS 3314, Nonferrous Metal Production",
        "NAICS 3361/3362, Automotive"
    ],
    'Data Type': df_original_features.dtypes.replace('float64', 'Numeric')
                                             .replace('int64', 'Numeric')
                                             .replace('object', 'Binary'),
    'min': df_original_features.min(),
    'max': df_original_features.max(),
    'mean': df_original_features.mean(),
    'std': df_original_features.std(),
    'Number of counties': df_original_features.select_dtypes(include=['number'])
                                              .apply(lambda x: (x > 0).sum(), axis=0)
}).reset_index(drop=True)

# Replace NaNs with 'N/A' in min, max, mean, std
summary[['min', 'max', 'mean', 'std']] = summary[['min', 'max', 'mean', 'std']].fillna('N/A')

# Convert min/max to int if possible
summary['max'] = summary['max'].apply(lambda x: int(x) if isinstance(x, (int, float)) else x)
summary['min'] = summary['min'].apply(lambda x: int(x) if isinstance(x, (int, float)) else x)

# Fill missing count values with fallback and convert to integer
summary['Number of counties'] = summary['Number of counties'].fillna(3233).astype(int)

# Format mean and std to 1 decimal place
summary['mean'] = summary['mean'].apply(lambda x: f"{x:.1f}" if isinstance(x, (int, float)) else x)
summary['std'] = summary['std'].apply(lambda x: f"{x:.1f}" if isinstance(x, (int, float)) else x)

# Adjust table display style
summary = summary.style.set_properties(**{'text-align': 'left'}) \
                       .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])

# Display styled summary table
summary


# Dimension Reduction

### Metrics

In [ ]:
def calculate_trustworthiness(X, X_embedded, n_neighbors=5):
    """Calculates the trustworthiness."""
    nn_original = NearestNeighbors(n_neighbors=n_neighbors)
    nn_embedded = NearestNeighbors(n_neighbors=n_neighbors)

    nn_original.fit(X)
    nn_embedded.fit(X_embedded)

    indices_original = nn_original.kneighbors(return_distance=False)
    indices_embedded = nn_embedded.kneighbors(return_distance=False)

    trust = 0.0
    for i in range(len(X)):
        U_i = set(indices_embedded[i])
        V_i = set(indices_original[i])
        K = len(U_i) 
        rank_sum = 0
        for j in U_i:
            if j not in V_i:
                original_distances = euclidean_distances(X[i:i+1], X)[0]
                sorted_indices = np.argsort(original_distances)
                rank = np.where(sorted_indices == j)[0][0] 
                rank_sum += (rank - K)
        trust += (1 - (2 / (K * (2 * len(X) - 3 * K - 1))) * rank_sum)
    return trust / len(X)


def calculate_continuity(X, X_embedded, n_neighbors=5):
    """Calculates the continuity."""
    # Continuity is essentially trustworthiness with spaces swapped
    return calculate_trustworthiness(X_embedded, X, n_neighbors=n_neighbors)


def calculate_stress1(original_distances, embedded_distances):
    """Calculates Stress-1 (Kruskal's Stress-1)."""
    numerator = np.sum((original_distances - embedded_distances)**2)
    denominator = np.sum(original_distances**2)
    return np.sqrt(numerator / denominator)

def calculate_shepard_diagram_metrics(original_data, embedded_data):
    """
    Calculates distances, Stress-1, and correlation for the Shepard diagram.
    Warning: This is computationally intensive for large datasets.
    """
    n_samples = original_data.shape[0]
    
    if n_samples > 5000:
        print("Warning: Calculation of all-pairs distances is very computationally intensive for over 5000 samples. Skipping Stress-1 and Shepard correlation.")
        return None, None

    original_dist_matrix = euclidean_distances(original_data)
    embedded_dist_matrix = euclidean_distances(embedded_data)

    triu_indices = np.triu_indices(n_samples, k=1)
    original_distances_flat = original_dist_matrix[triu_indices]
    embedded_distances_flat = embedded_dist_matrix[triu_indices]

    stress1 = calculate_stress1(original_distances_flat, embedded_distances_flat)
    
    shepard_correlation, _ = spearmanr(original_distances_flat, embedded_distances_flat)
    
    return stress1, shepard_correlation

### Parameter Grids

In [ ]:
# --- Grid Search Parameters ---

# t-SNE Parameters
tsne_perplexities = np.arange(5, 51, 5) # 5, 10, ..., 70
tsne_learning_rates = np.arange(100, 301, 100) # 100, 200, ..., 500

# UMAP Parameters
umap_n_neighbors = np.arange(10, 51, 5) # 10, 20, ..., 100
# min_dist starting from 0.1 in 20 steps, evenly distributed
umap_min_dist_start = 0.1
umap_min_dist_end = 0.1 + 19 * 0.01 # 20 steps, so 19 intervals of 0.01
umap_min_distances = np.linspace(umap_min_dist_start, umap_min_dist_end, 20).round(3) # Round to 3 decimal places
# Overriding the above linspace generation with a specific list of values
umap_min_distances = np.array([0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5])

# Storage Paths
output_dir = "data/processed_data_pkl_embedding_grid_search_results"
tsne_output_dir = os.path.join(output_dir, "tsne_results")
umap_output_dir = os.path.join(output_dir, "umap_results")

# Create output directories if they don't exist
os.makedirs(tsne_output_dir, exist_ok=True)
os.makedirs(umap_output_dir, exist_ok=True)

### t-SNE Gridsearch

In [ ]:
def perform_tsne_grid_search(dataframe_scaled, tsne_perplexities, tsne_learning_rates, tsne_output_dir):
    """
    Performs a grid search for t-SNE embeddings, evaluating different perplexity and learning rate values.

    Args:
        dataframe_scaled (pd.DataFrame): The scaled input data for t-SNE.
        tsne_perplexities (list): A list of perplexity values to test.
        tsne_learning_rates (list): A list of learning rate values to test.
        tsne_output_dir (str): The directory to save the results for each perplexity.

    Returns:
        list: A list of dictionaries, where each dictionary contains the results
              (including metrics) for a specific t-SNE configuration.
    """
    # --- t-SNE Grid Search ---
    print("\nStarting t-SNE Grid Search...")
    tsne_results = []

    # Calculate total iterations for the progress bar
    total_iterations = len(tsne_perplexities) * len(tsne_learning_rates)

    with tqdm(total=total_iterations, desc="t-SNE Grid Search") as pbar:
        for perplexity in tsne_perplexities:
            current_perplexity_results = []
            # The perplexity processing message can remain, as it's more specific than the overall progress bar
            print(f"\nProcessing t-SNE Perplexity: {perplexity}") 
            for lr in tsne_learning_rates:
                # The learning rate message can also remain for detailed logging
                print(f"  Learning Rate: {lr}")
                try:
                    tsne = TSNE(n_components=2, perplexity=perplexity, learning_rate=lr,
                                random_state=42, n_jobs=-1,  # Utilize all available CPU cores if possible
                                init='pca')  # PCA initialization is often more stable
                    tsne_embedding = tsne.fit_transform(dataframe_scaled)

                    # Calculate metrics
                    # These functions (calculate_trustworthiness, calculate_continuity, calculate_shepard_diagram_metrics)
                    # are assumed to be defined elsewhere and accessible.
                    trustworthiness = calculate_trustworthiness(dataframe_scaled.values, tsne_embedding)
                    continuity = calculate_continuity(dataframe_scaled.values, tsne_embedding)
                    
                    # Shepard Diagram Metrics (Stress-1, Correlation)
                    stress1, shepard_correlation = calculate_shepard_diagram_metrics(dataframe_scaled.values, tsne_embedding)

                    current_perplexity_results.append({
                        'algorithm': 't-SNE',
                        'perplexity': perplexity,
                        'learning_rate': lr,
                        'embedding_shape': tsne_embedding.shape,
                        'trustworthiness': trustworthiness,
                        'continuity': continuity,
                        'stress_1': stress1,
                        'shepard_correlation': shepard_correlation
                    })
                    print(f"    Trustworthiness: {trustworthiness:.4f}, Stress-1: {stress1:.4f}")

                except Exception as e:
                    print(f"Error during t-SNE (perplexity={perplexity}, lr={lr}): {e}")
                    current_perplexity_results.append({
                        'algorithm': 't-SNE',
                        'perplexity': perplexity,
                        'learning_rate': lr,
                        'embedding_shape': None,
                        'trustworthiness': None,
                        'continuity': None,
                        'stress_1': None,
                        'shepard_correlation': None,
                        'error': str(e)
                    })
                gc.collect()  # Release memory
                pbar.update(1) # Update the progress bar after each iteration

            # Save results for the current perplexity
            tsne_perplexity_df = pd.DataFrame(current_perplexity_results)
            file_name = f"tsne_perplexity_{perplexity}.pkl"
            file_path = os.path.join(tsne_output_dir, file_name)
            with open(file_path, 'wb') as f:
                pickle.dump(tsne_perplexity_df, f)
            print(f"  Results for Perplexity {perplexity} saved to {file_path}")
            tsne_results.extend(current_perplexity_results)  # For the final summary
            
    return tsne_results

In [ ]:
df_scaled

In [ ]:
tsne_results = perform_tsne_grid_search(df_scaled.iloc[:, 1:-4], tsne_perplexities, tsne_learning_rates, tsne_output_dir)
print("\nAll t-SNE results collected:")


### UMAP Gridsearch

In [ ]:
def perform_umap_grid_search(dataframe_scaled, umap_n_neighbors, umap_min_distances, umap_output_dir):
    """
    Performs a grid search for UMAP embeddings, evaluating different n_neighbors and min_dist values.

    Args:
        dataframe_scaled (pd.DataFrame): The scaled input data for UMAP.
        umap_n_neighbors (list): A list of n_neighbors values to test.
        umap_min_distances (list): A list of min_dist values to test.
        umap_output_dir (str): The directory to save the results for each n_neighbors.

    Returns:
        list: A list of dictionaries, where each dictionary contains the results
              (including metrics) for a specific UMAP configuration.
    """
    # --- UMAP Grid Search ---
    print("\nStarting UMAP Grid Search...")
    umap_results = []

    # Calculate total iterations for the progress bar
    total_iterations = len(umap_n_neighbors) * len(umap_min_distances)

    with tqdm(total=total_iterations, desc="UMAP Grid Search") as pbar:
        for n_neighbors in umap_n_neighbors:
            current_n_neighbors_results = []
            # The n_neighbors processing message can remain, as it's more specific than the overall progress bar
            print(f"\nProcessing UMAP n_neighbors: {n_neighbors}")
            for min_dist in umap_min_distances:
                # The minimum distance message can also remain for detailed logging
                print(f"  Minimum Distance: {min_dist}")
                try:
                    umap_model = UMAP(n_components=2, n_neighbors=n_neighbors, min_dist=min_dist,
                                      random_state=42, n_jobs=-1) # Utilize all available CPU cores
                    umap_embedding = umap_model.fit_transform(dataframe_scaled)

                    # Calculate metrics
                    # These functions (calculate_trustworthiness, calculate_continuity, calculate_shepard_diagram_metrics)
                    # are assumed to be defined elsewhere and accessible.
                    trustworthiness = calculate_trustworthiness(dataframe_scaled.values, umap_embedding)
                    continuity = calculate_continuity(dataframe_scaled.values, umap_embedding)
                    
                    # Shepard Diagram Metrics
                    stress1, shepard_correlation = calculate_shepard_diagram_metrics(dataframe_scaled.values, umap_embedding)

                    current_n_neighbors_results.append({
                        'algorithm': 'UMAP',
                        'n_neighbors': n_neighbors,
                        'min_dist': min_dist,
                        'embedding_shape': umap_embedding.shape,
                        'trustworthiness': trustworthiness,
                        'continuity': continuity,
                        'stress_1': stress1,
                        'shepard_correlation': shepard_correlation
                    })
                    print(f"    Trustworthiness: {trustworthiness:.4f}, Stress-1: {stress1:.4f}")

                except Exception as e:
                    print(f"Error during UMAP (n_neighbors={n_neighbors}, min_dist={min_dist}): {e}")
                    current_n_neighbors_results.append({
                        'algorithm': 'UMAP',
                        'n_neighbors': n_neighbors,
                        'min_dist': min_dist,
                        'embedding_shape': None,
                        'trustworthiness': None,
                        'continuity': None,
                        'stress_1': None,
                        'shepard_correlation': None,
                        'error': str(e)
                    })
                gc.collect() # Release memory
                pbar.update(1) # Update the progress bar after each iteration

            # Save results for the current n_neighbors
            umap_n_neighbors_df = pd.DataFrame(current_n_neighbors_results)
            file_name = f"umap_n_neighbors_{n_neighbors}.pkl"
            file_path = os.path.join(umap_output_dir, file_name)
            with open(file_path, 'wb') as f:
                pickle.dump(umap_n_neighbors_df, f)
            print(f"  Results for n_neighbors {n_neighbors} saved to {file_path}")
            umap_results.extend(current_n_neighbors_results) # For the final summary
            
    return umap_results

In [ ]:
#umap_results = perform_umap_grid_search(df_scaled.iloc[:, 1:-4], umap_n_neighbors, umap_min_distances, umap_output_dir)
#print("\nAll t-SNE results collected:")

### Loading Results

In [ ]:
# Define output directories (assuming these are already defined as in your context)
output_dir = "data/processed_data/pkl/embedding_grid_search_results"
tsne_output_dir = os.path.join(output_dir, "tsne_results")
umap_output_dir = os.path.join(output_dir, "umap_results")


# --- Consolidate Results ---
print("\nConsolidating all results...")

# List to store individual t-SNE DataFrames
list_tsne_dfs = []
# Iterate through files in the t-SNE output directory
for file_name in os.listdir(tsne_output_dir):
    # Process only pickle files
    if file_name.endswith(".pkl"):
        file_path = os.path.join(tsne_output_dir, file_name)
        # Open and load each pickle file
        with open(file_path, 'rb') as f:
            df_part = pickle.load(f)
            # Only append if the DataFrame is not empty
            if not df_part.empty:
                list_tsne_dfs.append(df_part)

# Concatenate all t-SNE DataFrames from the list at once
if list_tsne_dfs: # Only concatenate if the list of non-empty DataFrames is not empty
    final_tsne_df = pd.concat(list_tsne_dfs, ignore_index=True)
else:
    # Define columns explicitly if starting with an empty DataFrame
    # This assumes all your results DataFrames have these columns.
    # Adjust this list based on your actual column names.
    default_columns = [
        'algorithm', 'perplexity', 'learning_rate', 'embedding_shape',
        'trustworthiness', 'continuity', 'stress_1', 'shepard_correlation', 'error',
        'n_neighbors', 'min_dist' # Include UMAP specific columns as well for overall compatibility
    ]
    final_tsne_df = pd.DataFrame(columns=default_columns) # Initialize with known columns

# List to store individual UMAP DataFrames
list_umap_dfs = []
# Iterate through files in the UMAP output directory
for file_name in os.listdir(umap_output_dir):
    # Process only pickle files
    if file_name.endswith(".pkl"):
        file_path = os.path.join(umap_output_dir, file_name)
        # Open and load each pickle file
        with open(file_path, 'rb') as f:
            df_part = pickle.load(f)
            # Only append if the DataFrame is not empty
            if not df_part.empty:
                list_umap_dfs.append(df_part)

# Concatenate all UMAP Dataframes from the list at once
if list_umap_dfs: # Only concatenate if the list of non-empty DataFrames is not empty
    final_umap_df = pd.concat(list_umap_dfs, ignore_index=True)
else:
    # Define columns explicitly if starting with an empty DataFrame
    default_columns = [
        'algorithm', 'perplexity', 'learning_rate', 'embedding_shape',
        'trustworthiness', 'continuity', 'stress_1', 'shepard_correlation', 'error',
        'n_neighbors', 'min_dist'
    ]
    final_umap_df = pd.DataFrame(columns=default_columns) # Initialize with known columns


# Concatenate t-SNE and UMAP results into a single final DataFrame
# This specific concat operation is the one that might still warn if one of them is empty.
# By ensuring they start with defined columns if empty, it helps.
final_results_df = pd.concat([final_tsne_df, final_umap_df], ignore_index=True)

# Define path for the final consolidated output file
final_output_file = os.path.join(output_dir, "all_embedding_results.pkl")
# Save the final DataFrame to a pickle file
with open(final_output_file, 'wb') as f:
    pickle.dump(final_results_df, f)

print(f"\nAll results successfully consolidated and saved to {final_output_file}.")
print("\nFinal DataFrame (first 5 rows):")
# Display the first 5 rows of the final DataFrame
try:
    display(final_results_df.head())
except NameError:
    print(final_results_df.head())

print(f"\nNumber of t-SNE results: {len(final_tsne_df)}")
print(f"Number of UMAP results: {len(final_umap_df)}")

In [ ]:
# --- Configuration ---
output_dir = "data/processed_data/pkl/embedding_grid_search_results"
final_output_file = os.path.join(output_dir, "all_embedding_results.pkl")

# --- Load Results ---
try:
    with open(final_output_file, 'rb') as f:
        results_df = pickle.load(f)
    print(f"Results successfully loaded from '{final_output_file}'.")
except FileNotFoundError:
    print(f"Error: The file '{final_output_file}' was not found. Please ensure the previous step has been executed.")
    exit()


print(f"Total number of entries: {len(results_df)}")

# --- Clean up failed runs ---
# Remove rows where 'embedding_shape' is None (indicator for errors)
results_df_cleaned = results_df.dropna(subset=['embedding_shape', 'trustworthiness', 'continuity', 'shepard_correlation'])

# --- Calculate Ranking and Mean Rank ---

# Metrics where higher values are better (ascending=False for ranking)
positive_metrics = ['trustworthiness', 'continuity', 'shepard_correlation']
# Metrics where lower values are better (ascending=True for ranking)
negative_metrics = ['stress_1']

# DataFrame for ranks
ranked_results_df = results_df_cleaned.copy()

# Ranks for t-SNE
tsne_df = ranked_results_df[ranked_results_df['algorithm'] == 't-SNE'].copy()
for metric in positive_metrics:
    # Note that 'stress_1' should not be included here, which is automatically handled as it's not in positive_metrics
    tsne_df[f'{metric}_rank'] = tsne_df[metric].rank(method='average', ascending=False)
for metric in negative_metrics:
    # Stress-1 is usually ranked here, but not included in the mean rank for t-SNE
    tsne_df[f'{metric}_rank'] = tsne_df[metric].rank(method='average', ascending=True)

# Columns for the mean rank for t-SNE
# Explicitly exclude metrics that should not be considered for t-SNE's mean rank
tsne_metrics_for_mean_rank = [f'{m}_rank' for m in positive_metrics] + \
                             [f'{m}_rank' for m in negative_metrics] 
                             
tsne_df['mean_rank'] = tsne_df[tsne_metrics_for_mean_rank].mean(axis=1)

# Ranks for UMAP
umap_df = ranked_results_df[ranked_results_df['algorithm'] == 'UMAP'].copy()
for metric in positive_metrics:
    umap_df[f'{metric}_rank'] = umap_df[metric].rank(method='average', ascending=False)
for metric in negative_metrics:
    umap_df[f'{metric}_rank'] = umap_df[metric].rank(method='average', ascending=True)

# Columns for the mean rank for UMAP (all metrics are considered here)
umap_metrics_for_mean_rank = [f'{m}_rank' for m in positive_metrics] + \
                             [f'{m}_rank' for m in negative_metrics]

umap_df['mean_rank'] = umap_df[umap_metrics_for_mean_rank].mean(axis=1)

# --- Find Best Parameters ---

print("\n--- Best Parameters for t-SNE ---")
best_tsne_params = tsne_df.sort_values(by='mean_rank').iloc[0]
print(f"Best t-SNE Parameters (Mean Rank: {best_tsne_params['mean_rank']:.2f}):")
print(f"  Perplexity: {best_tsne_params['perplexity']}")
print(f"  Learning Rate: {best_tsne_params['learning_rate']}")
print("  Metrics for this combination:")
print(f"    Trustworthiness: {best_tsne_params['trustworthiness']:.4f} (Rank: {best_tsne_params['trustworthiness_rank']:.2f})")
print(f"    Continuity: {best_tsne_params['continuity']:.4f} (Rank: {best_tsne_params['continuity_rank']:.2f})")
print(f"    Shepard Correlation: {best_tsne_params['shepard_correlation']:.4f} (Rank: {best_tsne_params['shepard_correlation_rank']:.2f})")
print(f"    Stress-1 (not used for ranking): {best_tsne_params['stress_1']:.4f}")


print("\n--- Best Parameters for UMAP ---")
best_umap_params = umap_df.sort_values(by='mean_rank').iloc[0]
print(f"Best UMAP Parameters (Mean Rank: {best_umap_params['mean_rank']:.2f}):")
print(f"  n_neighbors: {best_umap_params['n_neighbors']}")
print(f"  min_dist: {best_umap_params['min_dist']}")
print("  Metrics for this combination:")
print(f"    Trustworthiness: {best_umap_params['trustworthiness']:.4f} (Rank: {best_umap_params['trustworthiness_rank']:.2f})")
print(f"    Continuity: {best_umap_params['continuity']:.4f} (Rank: {best_umap_params['continuity_rank']:.2f})")
print(f"    Shepard Correlation: {best_umap_params['shepard_correlation']:.4f} (Rank: {best_umap_params['shepard_correlation_rank']:.2f})")
print(f"    Stress-1: {best_umap_params['stress_1']:.4f} (Rank: {best_umap_params['stress_1_rank']:.2f})")

In [ ]:
dataframe_scaled = df_scaled.iloc[:, 1:]
# --- Laden der vorherigen Grid Search Ergebnisse ---
output_dir = "data/processed_data/pkl/embedding_grid_search_results"
final_output_file = os.path.join(output_dir, "all_embedding_results.pkl")

try:
    with open(final_output_file, 'rb') as f:
        results_df = pickle.load(f)
except FileNotFoundError:
    print(f"Fehler: Die Datei '{final_output_file}' wurde nicht gefunden. Bitte stellen Sie sicher, dass der vorherige Schritt ausgeführt wurde.")
    exit()

# --- Bereinigung der Ergebnisse und Bestimmung der besten t-SNE und UMAP Parameter ---
results_df_cleaned = results_df.dropna(subset=['embedding_shape', 'trustworthiness', 'continuity', 'shepard_correlation'])

# Metriken, bei denen höhere Werte besser sind (ascending=False für Ranking)
positive_metrics = ['trustworthiness', 'continuity', 'shepard_correlation']
# Metriken, bei denen niedrigere Werte besser sind (ascending=True für Ranking)
negative_metrics = ['stress_1']

# t-SNE besten Parameter finden
tsne_df = results_df_cleaned[results_df_cleaned['algorithm'] == 't-SNE'].copy()
for metric in positive_metrics:
    tsne_df[f'{metric}_rank'] = tsne_df[metric].rank(method='average', ascending=False)
for metric in negative_metrics: # Stress-1 wird hier gerankt, aber nicht in den mittleren Rang einbezogen
    tsne_df[f'{metric}_rank'] = tsne_df[metric].rank(method='average', ascending=True)

tsne_metrics_for_mean_rank = [f'{m}_rank' for m in positive_metrics] + \
                             [f'{m}_rank' for m in negative_metrics if m != 'stress_1'] # Stress-1 ausschließen
tsne_df['mean_rank'] = tsne_df[tsne_metrics_for_mean_rank].mean(axis=1)
best_tsne_params = tsne_df.sort_values(by='mean_rank').iloc[0]

# UMAP besten Parameter finden
umap_df = results_df_cleaned[results_df_cleaned['algorithm'] == 'UMAP'].copy()
for metric in positive_metrics:
    umap_df[f'{metric}_rank'] = umap_df[metric].rank(method='average', ascending=False)
for metric in negative_metrics:
    umap_df[f'{metric}_rank'] = umap_df[metric].rank(method='average', ascending=True)

umap_metrics_for_mean_rank = [f'{m}_rank' for m in positive_metrics] + \
                             [f'{m}_rank' for m in negative_metrics] # Alle Metriken einbeziehen
umap_df['mean_rank'] = umap_df[umap_metrics_for_mean_rank].mean(axis=1)
best_umap_params = umap_df.sort_values(by='mean_rank').iloc[0]

# --- PCA Berechnung und Metriken ---
print("\nBerechne PCA-Einbettung und Metriken...")
pca = PCA(n_components=2, random_state=42)
pca_embedding = pca.fit_transform(dataframe_scaled)

pca_trustworthiness = calculate_trustworthiness(dataframe_scaled.values, pca_embedding)
pca_continuity = calculate_continuity(dataframe_scaled.values, pca_embedding)
pca_stress1, pca_shepard_correlation = calculate_shepard_diagram_metrics(dataframe_scaled.values, pca_embedding)

pca_results = {
    'algorithm': 'PCA',
    'trustworthiness': pca_trustworthiness,
    'continuity': pca_continuity,
    'stress_1': pca_stress1,
    'shepard_correlation': pca_shepard_correlation
}

# --- Finale Tabelle erstellen ---
final_table_data = []

# Daten für PCA
final_table_data.append({
    'Method': 'PCA',
    'Trustworthiness': pca_results['trustworthiness'],
    'Continuity': pca_results['continuity'],
    'Shepard Correlation': pca_results['shepard_correlation'],
    'Stress-1': pca_results['stress_1']
})

# Daten für t-SNE (beste Parameter)
final_table_data.append({
    'Method': 't-SNE',
    'Trustworthiness': best_tsne_params['trustworthiness'],
    'Continuity': best_tsne_params['continuity'],
    'Shepard Correlation': best_tsne_params['shepard_correlation'],
    'Stress-1': best_tsne_params['stress_1']
})

# Daten für UMAP (beste Parameter)
final_table_data.append({
    'Method': 'UMAP',
    'Trustworthiness': best_umap_params['trustworthiness'],
    'Continuity': best_umap_params['continuity'],
    'Shepard Correlation': best_umap_params['shepard_correlation'],
    'Stress-1': best_umap_params['stress_1']
})

final_comparison_df = pd.DataFrame(final_table_data)

print("\n--- Finale Vergleichstabelle der besten Parameter für jede Methode ---")
print(final_comparison_df.round(4).to_string(index=False))


In [ ]:
# 1. Calculate Embeddings
X_pca = PCA(n_components=2, random_state=42).fit_transform(df_scaled_features)
X_tsne = TSNE(n_components=2,
              perplexity=35,
              learning_rate=200,
              random_state=42).fit_transform(df_scaled_features)
X_umap = UMAP(n_components=2,
              n_neighbors=20,
              min_dist=0.4,
              random_state=42).fit_transform(df_scaled_features)

In [ ]:
# --- 2. Subplots erstellen ---
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# Variables for plot settings
s = 2         # Point size
legend_fontsize = 18 # Font size for the legend text

# --- PCA-Plot ---
axes[0].scatter(X_pca[:, 0], X_pca[:, 1], s=s, color='black', alpha=0.7)
# Remove title, add legend
axes[0].legend(["PCA"], loc='lower right', fontsize=legend_fontsize, frameon=False)
axes[0].set_xticks([])
axes[0].set_yticks([])

# --- t-SNE-Plot ---
axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], s=s, color='black', alpha=0.7)
# Remove title, add legend with parameters
axes[1].legend(["t-SNE"], loc='lower right', fontsize=legend_fontsize, frameon=False)
axes[1].set_xticks([])
axes[1].set_yticks([])

# --- UMAP-Plot ---
axes[2].scatter(X_umap[:, 0], X_umap[:, 1], s=s, color='black', alpha=0.7)
# Remove title, add legend with parameters
axes[2].legend(["UMAP"], loc='lower right', fontsize=legend_fontsize, frameon=False)
axes[2].set_xticks([])
axes[2].set_yticks([])

plt.tight_layout(pad=0.0)

# --- Save Image ---
# Save the image with 600 DPI
plt.savefig("dimensionality_reduction_plots.png", dpi=600, bbox_inches='tight')


#plt.suptitle("Dimensionality Reduction Techniques Comparison", fontsize=16, y=1.02)
plt.show()

# Clustering

## HDBScan (full feature space)

In [ ]:
# Assuming df_scaled is already defined and loaded
data_scaled = df_scaled.iloc[:, 1:].values # .values to ensure it's a NumPy array

# --- HDBSCAN Parameter Grid ---
hdbscan_params = {
    'min_cluster_size': range(10, 50, 2),  # Example values
    'min_samples': range(1, 20, 1)        # Example values
}

# --- Initialization for Results ---
hdbscan_raw_results = [] # To store all RAW results for analysis

# --- Perform Grid Search ---
total_iterations = len(hdbscan_params['min_cluster_size']) * len(hdbscan_params['min_samples'])
with tqdm(total=total_iterations, desc="HDBSCAN Grid Search (Phase 1: Data Collection)") as pbar:
    for min_cluster_size in hdbscan_params['min_cluster_size']:
        for min_samples in hdbscan_params['min_samples']:
            hdbscan_model = hdbscan.HDBSCAN(
                min_cluster_size=int(min_cluster_size),
                min_samples=int(min_samples),
                cluster_selection_method='eom', # Explicitly 'eom' for cluster_persistence_
                allow_single_cluster=True # Allows all non-noise points to be in a single cluster
            )
            labels = hdbscan_model.fit_predict(data_scaled)

            # Determine the number of unique clusters (excluding noise)
            unique_clusters = len(set(labels) - {-1})

            # Consider only points assigned to a cluster (not noise)
            clustered_points_indices = labels != -1

            current_silhouette = -1.0
            current_negative_silhouette_ratio = 1.0
            current_hdbscan_persistence = 0.0 # Default value if not calculable

            # Check: At least 2 assigned points AND at least 2 clusters for Silhouette Score
            if np.sum(clustered_points_indices) >= 2 and unique_clusters >= 2:
                current_silhouette = silhouette_score(data_scaled[clustered_points_indices], labels[clustered_points_indices])
                
                silhouette_vals = silhouette_samples(data_scaled[clustered_points_indices], labels[clustered_points_indices])
                current_negative_silhouette_ratio = np.sum(silhouette_vals < 0) / len(silhouette_vals)
            
            # cluster_persistence_ is only calculated if cluster_selection_method='eom' (default)
            # and if clusters were found. If only noise or a single cluster, it is None or empty.
            if hdbscan_model.cluster_persistence_ is not None and len(hdbscan_model.cluster_persistence_) > 0:
                current_hdbscan_persistence = np.mean(hdbscan_model.cluster_persistence_)
            else:
                current_hdbscan_persistence = 0.0 # Set to 0 if persistence not calculable

            # Noise statistics
            noise_count = np.sum(labels == -1)
            current_noise_ratio = noise_count / len(labels)

            # Store results (still WITHOUT combined score, as persistence not yet normalized)
            hdbscan_raw_results.append({
                'min_cluster_size': int(min_cluster_size),
                'min_samples': int(min_samples),
                'unique_clusters': unique_clusters,
                'silhouette_avg': current_silhouette,
                'negative_silhouette_ratio': current_negative_silhouette_ratio,
                'noise_ratio': current_noise_ratio,
                'hdbscan_persistence': current_hdbscan_persistence # Store raw value
            })
            
            pbar.update(1) # Update progress bar

# Convert raw data to a DataFrame
hdbscan_results_df = pd.DataFrame(hdbscan_raw_results)
hdbscan_results_df.to_pickle('data/processed_data/pkl/cluster_results/hdbscan_grid_search_results.pickle')
print("\nAll Grid Search results saved to 'data/processed_data/pkl/cluster_results/hdbscan_grid_search_results.pickle'")

## HDBScan (t-SNE)

In [ ]:
data_scaled = X_tsne

# --- HDBSCAN Parameter Grid ---
hdbscan_params = {
    'min_cluster_size': range(10, 50, 5),  # Example values
    'min_samples': range(1, 20, 1)        # Example values
}

# --- Initialization for Raw Results ---
hdbscan_raw_results = [] # To store all RAW results for analysis

# --- Perform Grid Search (Phase 1) ---
total_iterations = len(hdbscan_params['min_cluster_size']) * len(hdbscan_params['min_samples'])
with tqdm(total=total_iterations, desc="HDBSCAN Grid Search (Phase 1: Data Collection)") as pbar:
    for min_cluster_size in hdbscan_params['min_cluster_size']:
        for min_samples in hdbscan_params['min_samples']:
            hdbscan_model = hdbscan.HDBSCAN(
                min_cluster_size=int(min_cluster_size),
                min_samples=int(min_samples),
                cluster_selection_method='eom', # Explicitly 'eom' for cluster_persistence_
                allow_single_cluster=True        # Allows all non-noise points to be in a single cluster
            )
            labels = hdbscan_model.fit_predict(data_scaled)

            # Determine the number of unique clusters (excluding noise)
            unique_clusters = len(set(labels) - {-1})

            # Consider only points assigned to a cluster (not noise)
            clustered_points_indices = labels != -1

            # Initialize metrics with default values for cases where they are not calculable
            current_silhouette = -1.0 # Very poor if not calculable
            current_negative_silhouette_ratio = 1.0 # All "negative" if not calculable
            current_hdbscan_persistence = 0.0 # No persistence if not calculable

            # Calculate metrics if sufficient clusters and points are available
            if np.sum(clustered_points_indices) >= 2 and unique_clusters >= 2:
                current_silhouette = silhouette_score(data_scaled[clustered_points_indices], labels[clustered_points_indices])
                
                silhouette_vals = silhouette_samples(data_scaled[clustered_points_indices], labels[clustered_points_indices])
                current_negative_silhouette_ratio = np.sum(silhouette_vals < 0) / len(silhouette_vals)
            
            # Calculate cluster persistence (only if 'eom' and clusters exist)
            if hdbscan_model.cluster_persistence_ is not None and len(hdbscan_model.cluster_persistence_) > 0:
                current_hdbscan_persistence = np.mean(hdbscan_model.cluster_persistence_)
            # If no clusters or only noise/a single cluster, hdbscan_persistence_ remains 0.0

            # Noise statistics
            noise_count = np.sum(labels == -1)
            current_noise_ratio = noise_count / len(labels)

            # Store results (Raw values, NO combined score in this phase)
            hdbscan_raw_results.append({
                'min_cluster_size': int(min_cluster_size),
                'min_samples': int(min_samples),
                'unique_clusters': unique_clusters,
                'silhouette_avg': current_silhouette,
                'negative_silhouette_ratio': current_negative_silhouette_ratio,
                'noise_ratio': current_noise_ratio,
                'hdbscan_persistence': current_hdbscan_persistence # Store raw value
            })
            
            pbar.update(1) # Update progress bar

# Convert the collected raw data into a DataFrame
hdbscan_results_df = pd.DataFrame(hdbscan_raw_results)
# Save the complete results
hdbscan_results_df.to_pickle('data/processed_data/pkl/cluster_results/hdbscan_grid_search_results_tsne.pickle')
print("\nAll Grid Search results saved to 'data/processed_data/pkl/cluster_results/hdbscan_grid_search_results_tsne.pickle'")

In [ ]:
def calculate_combined_score_normalized(
    silhouette_avg,
    negative_silhouette_ratio,
    noise_ratio,
    hdbscan_persistence,
    min_overall_persistence, # Global min value for normalization
    max_overall_persistence, # Global max value for normalization
    noise_threshold=0.10, # 10%
    W1=1.0, # Weight for normalized average silhouette score (Maximize)
    W2=0.5, # Weight for normalized negative silhouette ratio (Minimize)
    W3=1.5, # Weight for normalized HDBSCAN persistence (Maximize)
    W4=5.0  # High penalty for exceeding noise threshold
):
    """
    Calculates a combined score for optimizing HDBSCAN parameters after normalization.
    Uses Min-Max normalization for hdbscan_persistence based on global min/max values.

    Parameters:
    - silhouette_avg (float): Average silhouette score.
    - negative_silhouette_ratio (float): Ratio of negative silhouette scores.
    - noise_ratio (float): Ratio of noise points.
    - hdbscan_persistence (float): HDBSCAN persistence value.
    - min_overall_persistence (float): Global minimum persistence value for normalization.
    - max_overall_persistence (float): Global maximum persistence value for normalization.
    - noise_threshold (float): Maximum acceptable noise ratio before applying a penalty.
    - W1, W2, W3, W4 (float): Weights for the different metrics in the combined score.

    Returns:
    - float: The calculated combined score.
    """
    # --- Normalize Metrics to the Range [0, 1] ---

    # 1. silhouette_avg: Range [-1, 1] -> [0, 1]
    # A score of -1 becomes 0, a score of 1 becomes 1.
    normalized_silhouette_avg = (silhouette_avg + 1) / 2

    # 2. negative_silhouette_ratio: Already in range [0, 1]
    normalized_negative_silhouette_ratio = negative_silhouette_ratio

    # 3. noise_ratio: Already in range [0, 1]
    normalized_noise_ratio = noise_ratio

    # 4. hdbscan_persistence: Min-Max Normalization
    # Ensure division by zero is avoided if max and min are equal (e.g., all persistences are 0)
    if max_overall_persistence > min_overall_persistence:
        normalized_hdbscan_persistence = (hdbscan_persistence - min_overall_persistence) / \
                                         (max_overall_persistence - min_overall_persistence)
    else: # All persistence values are equal (e.g., all 0) or only one value
        normalized_hdbscan_persistence = 0.0 # No variation, thus no positive contribution from persistence

    # --- Calculate Combined Score ---
    # Maximize silhouette_avg and hdbscan_persistence (positive weights)
    # Minimize negative_silhouette_ratio (negative weight)
    combined_score = W1 * normalized_silhouette_avg
    combined_score -= W2 * normalized_negative_silhouette_ratio
    combined_score += W3 * normalized_hdbscan_persistence

    # Noise ratio: High penalty if threshold is exceeded
    if normalized_noise_ratio > noise_threshold:
        # The penalty is proportional to how much the threshold is exceeded
        combined_score -= W4 * (normalized_noise_ratio - noise_threshold) * 10

    return combined_score

def process_hdbscan_grid_search_results(hdbscan_results_df: pd.DataFrame, output_filename: str):
    """
    Processes the results of an HDBSCAN grid search, calculates normalized combined scores,
    identifies the best parameters, and saves the enhanced DataFrame.

    Parameters:
    - hdbscan_results_df (pd.DataFrame): DataFrame containing the raw HDBSCAN grid search results.
                                         Expected columns: 'silhouette_avg', 'negative_silhouette_ratio',
                                         'noise_ratio', 'hdbscan_persistence', 'min_cluster_size',
                                         'min_samples', 'unique_clusters'.
    - output_filename (str): The full path including filename to save the processed DataFrame.
    """
    print(f"\nStarting processing for: {output_filename}")

    # --- Step 1: Determine Global Min/Max Values for Persistence ---
    # Filter persistence values that are not 0 to make the normalization range meaningful.
    # If all persistence values are 0, we treat this specially.
    valid_persistences = hdbscan_results_df['hdbscan_persistence'][hdbscan_results_df['hdbscan_persistence'] > 0]

    if not valid_persistences.empty:
        min_overall_persistence = valid_persistences.min()
        max_overall_persistence = valid_persistences.max()
    else: # All persistence values are 0 (or no valid ones found)
        min_overall_persistence = 0.0
        max_overall_persistence = 0.0

    print(f"Phase 2: Calculating combined scores with normalized persistence.")
    print(f"Global Min Persistence (excluding 0): {min_overall_persistence:.4f}, Max Persistence: {max_overall_persistence:.4f}")

    # --- Step 2: Calculate Combined Scores and Find Best Score/Parameters ---
    best_score = -np.inf # Start with a very small value, as we want to maximize
    best_params = {}
    best_metrics = {}

    # Iterate over the collected results and calculate the combined score
    # tqdm provides a progress bar for the loop
    for index, row in tqdm(hdbscan_results_df.iterrows(), total=len(hdbscan_results_df), desc="Calculating final scores"):
        current_combined_score = calculate_combined_score_normalized(
            silhouette_avg=row['silhouette_avg'],
            negative_silhouette_ratio=row['negative_silhouette_ratio'],
            noise_ratio=row['noise_ratio'],
            hdbscan_persistence=row['hdbscan_persistence'],
            min_overall_persistence=min_overall_persistence,
            max_overall_persistence=max_overall_persistence
        )
        
        # Add the combined score as a new column to the DataFrame
        hdbscan_results_df.at[index, 'combined_score'] = current_combined_score

        # Update best score and parameters if the current score is better
        if current_combined_score > best_score:
            best_score = current_combined_score
            best_params = {
                'min_cluster_size': row['min_cluster_size'],
                'min_samples': row['min_samples']
            }
            best_metrics = {
                'silhouette_avg': row['silhouette_avg'],
                'negative_silhouette_ratio': row['negative_silhouette_ratio'],
                'noise_ratio': row['noise_ratio'],
                'hdbscan_persistence': row['hdbscan_persistence'],
                'unique_clusters': row['unique_clusters']
            }

    # --- Display and Save Results ---
    print("\n--- Grid Search Processing Completed ---")
    print(f"Best Parameters: {best_params}")
    print(f"Best Combined Score (normalized): {best_score:.4f}")
    print(f"Associated Metrics (Raw Values):")
    for metric, value in best_metrics.items():
        print(f"  {metric}: {value:.4f}")

    # Save the complete results to the specified pickle file
    hdbscan_results_df.to_pickle(output_filename)
    print(f"\nAll Grid Search results saved to '{output_filename}'")




In [ ]:
hdbscan_tsne_df = pd.read_pickle('data/processed_data/pkl/cluster_results/hdbscan_grid_search_results_tsne.pickle')
hdbscan_df = pd.read_pickle('data/processed_data/pkl/cluster_results/hdbscan_grid_search_results.pickle')

process_hdbscan_grid_search_results(
     hdbscan_results_df=hdbscan_df.copy(), # Use .copy() to avoid modifying original DF
     output_filename='data/processed_data/pkl/cluster_results/hdbscan_grid_search_results_normalized.pickle'
 )

print("\n" + "="*80 + "\n") # Separator for clarity

process_hdbscan_grid_search_results(
     hdbscan_results_df=hdbscan_tsne_df.copy(), # Use .copy() to avoid modifying original DF
     output_filename='data/processed_data/pkl/cluster_results/hdbscan_grid_search_results_tsne_normalized.pickle'
 )

In [ ]:
df_hdbscan = pickle.load(open('data/processed_data/pkl/cluster_results/hdbscan_grid_search_results_tsne_normalized.pickle', 'rb'))
df_hdbscan = df_hdbscan.sort_values(by='combined_score', ascending=False)
df_hdbscan

In [ ]:
df_hdbscan['min_cluster_size'].unique()

## DBScan (t-SNE)

In [ ]:
def run_dbscan_grid_search(data, eps_values, min_samples_values, output_filename='dbscan_grid_search_results.pickle'):
    """
    Performs a grid search for DBSCAN parameters (eps and min_samples) on the given data.

    Args:
        data (np.ndarray): The input data for DBSCAN clustering (e.g., t-SNE reduced data or scaled full feature space).
        eps_values (np.ndarray or list): An array or list of 'eps' values to test.
        min_samples_values (range or list): A range or list of 'min_samples' values to test.
        output_filename (str): The filename (including path) to save the results DataFrame as a pickle file.

    Returns:
        pd.DataFrame: A DataFrame containing the raw results of the grid search, including
                      'eps', 'min_samples', 'unique_clusters', 'silhouette_avg',
                      'negative_silhouette_ratio', and 'noise_ratio'.
    """
    dbscan_raw_results = []
    total_iterations = len(eps_values) * len(min_samples_values)

    with tqdm(total=total_iterations, desc="DBSCAN Grid Search (Phase 1: Datensammlung)") as pbar:
        for eps in eps_values:
            for min_samples in min_samples_values:
                dbscan_model = DBSCAN(
                    eps=float(eps),
                    min_samples=int(min_samples)
                )
                labels = dbscan_model.fit_predict(data)

                unique_clusters = len(set(labels) - {-1})
                clustered_points_indices = labels != -1
                
                current_silhouette = -1.0
                current_negative_silhouette_ratio = 1.0

                if unique_clusters >= 2 and np.sum(clustered_points_indices) >= 2:
                    current_silhouette = silhouette_score(data[clustered_points_indices], labels[clustered_points_indices])
                    
                    silhouette_vals = silhouette_samples(data[clustered_points_indices], labels[clustered_points_indices])
                    current_negative_silhouette_ratio = np.sum(silhouette_vals < 0) / len(silhouette_vals)
                
                noise_count = np.sum(labels == -1)
                current_noise_ratio = noise_count / len(labels)

                dbscan_raw_results.append({
                    'eps': float(eps),
                    'min_samples': int(min_samples),
                    'unique_clusters': unique_clusters,
                    'silhouette_avg': current_silhouette,
                    'negative_silhouette_ratio': current_negative_silhouette_ratio,
                    'noise_ratio': current_noise_ratio
                })
                
                pbar.update(1)

    dbscan_results_df = pd.DataFrame(dbscan_raw_results)
    dbscan_results_df.to_pickle(output_filename)
    print(f"\nAlle Grid Search Ergebnisse gespeichert unter '{output_filename}'")
    return dbscan_results_df

In [ ]:
data_reduced = X_tsne

# 2. Define your DBSCAN parameter ranges
eps_values_to_test = np.arange(0.5, 5.5, 0.1)
min_samples_values_to_test = range(5, 50, 1)

# 3. Define the output filename
output_file = 'data/processed_data/pkl/cluster_results/dbscan_grid_search_results_tsne.pickle'

# 4. Call the function
results_df = run_dbscan_grid_search(
    data=data_reduced,
    eps_values=eps_values_to_test,
    min_samples_values=min_samples_values_to_test,
    output_filename=output_file
)

# You can now work with the 'results_df' DataFrame
print("\nFirst 5 rows of the results DataFrame:")
print(results_df.head())

## DBScan (full feature space)

In [ ]:


def run_dbscan_grid_search_full_space(df_scaled, eps_values, min_samples_values, output_filename='dbscan_grid_search_results_full_space.pickle'):
    """
    Performs a grid search for DBSCAN parameters (eps and min_samples) on the full feature space data.

    Args:
        df_scaled (pd.DataFrame): The scaled DataFrame, where the first column is assumed to be an ID
                                   and subsequent columns are features.
        eps_values (np.ndarray or list): An array or list of 'eps' values to test.
        min_samples_values (range or list): A range or list of 'min_samples' values to test.
        output_filename (str): The filename (including path) to save the results DataFrame as a pickle file.

    Returns:
        pd.DataFrame: A DataFrame containing the raw results of the grid search, including
                      'eps', 'min_samples', 'unique_clusters', 'silhouette_avg',
                      'negative_silhouette_ratio', and 'noise_ratio'.
    """
    # Use .values for direct NumPy array input to DBSCAN, skipping the first column (e.g., ID).
    data_full_space = df_scaled.iloc[:, 1:].values

    dbscan_raw_results_full_space = []
    total_iterations = len(eps_values) * len(min_samples_values)

    with tqdm(total=total_iterations, desc="DBSCAN Grid Search (Phase 1: Data Collection - Full Feature Space)") as pbar:
        for eps in eps_values:
            for min_samples in min_samples_values:
                dbscan_model = DBSCAN(
                    eps=float(eps),
                    min_samples=int(min_samples)
                )
                labels = dbscan_model.fit_predict(data_full_space)

                # Filter out noise points (-1 label) for metric calculations
                clustered_points_indices = labels != -1
                
                # Check for valid clustering results for metric calculation
                unique_clusters = len(set(labels) - {-1})
                
                # Initialize metrics with default poor scores if not calculable
                current_silhouette = -1.0
                current_negative_silhouette_ratio = 1.0

                # Calculate metrics if there are at least 2 clusters and at least 2 clustered points
                if unique_clusters >= 2 and np.sum(clustered_points_indices) >= 2:
                    current_silhouette = silhouette_score(data_full_space[clustered_points_indices], labels[clustered_points_indices])
                    
                    silhouette_vals = silhouette_samples(data_full_space[clustered_points_indices], labels[clustered_points_indices])
                    current_negative_silhouette_ratio = np.sum(silhouette_vals < 0) / len(silhouette_vals)
                
                # Calculate noise statistics (always applicable)
                noise_count = np.sum(labels == -1)
                current_noise_ratio = noise_count / len(labels)

                # Store all raw results for later detailed analysis. Combined score calculated later.
                dbscan_raw_results_full_space.append({
                    'eps': float(eps),
                    'min_samples': int(min_samples),
                    'unique_clusters': unique_clusters, # Store raw value
                    'silhouette_avg': current_silhouette,
                    'negative_silhouette_ratio': current_negative_silhouette_ratio,
                    'noise_ratio': current_noise_ratio
                })
                
                pbar.update(1) # Update progress bar

    # Convert the collected raw data into a DataFrame
    dbscan_results_full_space_df = pd.DataFrame(dbscan_raw_results_full_space)
    # Save the complete results
    dbscan_results_full_space_df.to_pickle(output_filename)
    print(f"\nAlle Grid Search Ergebnisse gespeichert unter '{output_filename}'")
    return dbscan_results_full_space_df



In [ ]:
# 2. Define your DBSCAN parameter ranges
eps_values_to_test_full_space = np.arange(0.5, 5.0, 0.1)
min_samples_values_to_test_full_space = range(5, 50, 1)

# 3. Define the output filename
output_file_full_space = 'data/processed_data/pkl/cluster_results/dbscan_grid_search_results_full_space.pickle'

# 4. Call the function
results_df_full_space = run_dbscan_grid_search_full_space(
    df_scaled=df_scaled,
    eps_values=eps_values_to_test_full_space,
    min_samples_values=min_samples_values_to_test_full_space,
    output_filename=output_file_full_space
)

# You can now work with the 'results_df_full_space' DataFrame
print("\nFirst 5 rows of the full feature space results DataFrame:")
print(results_df_full_space.head())

In [ ]:
# --- Funktion zur Berechnung des kombinierten Scores (basierend auf Code 2 / Englisch) ---
def calculate_combined_score_dbscan_normalized_v2(
    silhouette_avg,
    negative_silhouette_ratio,
    noise_ratio,
    num_clusters,
    min_overall_clusters, # Global min value for num_clusters normalization
    max_overall_clusters, # Global max value for num_clusters normalization
    noise_threshold=0.10, # 10% maximum allowable noise ratio
    W1=1.0, # Weight for normalized average silhouette score (Maximize)
    W2=0.5, # Weight for normalized negative silhouette ratio (Minimize)
    W3_noise_penalty=5.0, # High penalty for exceeding noise threshold
    W4_cluster_count_bonus=1.0, # Bonus for cluster count in desired range
    target_min_clusters=2, # Lower bound of desired cluster range (e.g., at least 2 clusters)
    target_max_clusters=10 # Upper bound of desired cluster range (e.g., max 10 clusters)
):
    """
    Calculates a combined score to optimize DBSCAN parameters after normalization.
    Uses Min-Max normalization for num_clusters based on global min/max values.
    Applies bonus/penalty based on a target range for the number of clusters.
    This version applies a fixed bonus when num_clusters is in the target range.
    """
    # --- Normalization of Metrics to the [0, 1] range ---

    # 1. silhouette_avg: Range [-1, 1] -> [0, 1]
    normalized_silhouette_avg = (silhouette_avg + 1) / 2

    # 2. negative_silhouette_ratio: Already in [0, 1]
    normalized_negative_silhouette_ratio = negative_silhouette_ratio

    # 3. noise_ratio: Already in [0, 1]
    normalized_noise_ratio = noise_ratio

    # 4. num_clusters: Min-Max normalization based on global observed range
    normalized_num_clusters = 0.0
    if max_overall_clusters > min_overall_clusters:
        # If there's a range, normalize it
        normalized_num_clusters = (num_clusters - min_overall_clusters) / \
                                 (max_overall_clusters - min_overall_clusters)
    # If max_overall_clusters == min_overall_clusters (e.g., all 0 or all 1 cluster),
    # normalized_num_clusters remains 0.0, indicating no variability to contribute positively.

    # --- Calculate the Combined Score ---
    combined_score = W1 * normalized_silhouette_avg          # Maximize (good)
    combined_score -= W2 * normalized_negative_silhouette_ratio # Minimize (bad)

    # Noise ratio: High penalty if threshold is exceeded
    if normalized_noise_ratio > noise_threshold:
        combined_score -= W3_noise_penalty * (normalized_noise_ratio - noise_threshold) * 10

    # Bonus/Penalty for the number of clusters based on desired range
    if num_clusters >= target_min_clusters and num_clusters <= target_max_clusters:
        combined_score += W4_cluster_count_bonus # Fixed bonus for being in range
    else:
        # Outside the target range: apply a penalty
        if num_clusters < target_min_clusters:
            combined_score -= W4_cluster_count_bonus * (target_min_clusters - num_clusters) # Penalize for being too few
        elif num_clusters > target_max_clusters:
            combined_score -= W4_cluster_count_bonus * (num_clusters - target_max_clusters) # Penalize for being too many

    return combined_score

# --- Hilfsfunktion zur Durchführung der Score-Berechnung für einen DataFrame ---
def evaluate_dbscan_results(df, name, target_min_clusters, target_max_clusters):
    """
    Evaluates DBSCAN results for a given DataFrame using the scoring function.
    """
    print(f"\n--- Starting Evaluation for: {name} ---")

    # --- Step 2: Determine Global Min/Max for unique_clusters ---
    # Filter out 0 cluster counts to get a more meaningful min/max for normalization of "found" clusters.
    valid_unique_clusters = df['unique_clusters'][df['unique_clusters'] > 0]

    if not valid_unique_clusters.empty:
        min_overall_clusters = valid_unique_clusters.min()
        max_overall_clusters = valid_unique_clusters.max()
    else: # All cluster counts are 0 (or no valid ones found)
        min_overall_clusters = 0
        max_overall_clusters = 0

    print(f"Global Min Cluster Count (excluding 0): {min_overall_clusters}, Max Cluster Count: {max_overall_clusters}")

    # --- Initialize for Best Results ---
    best_score = -np.inf
    best_params = {}
    best_metrics = {}
    
    # Ensure 'combined_score' column exists
    df['combined_score'] = np.nan

    # --- Step 3: Calculate Combined Scores and Find Best Parameters ---
    for index, row in tqdm(df.iterrows(), total=len(df), desc=f"Calculating Scores for {name}"):
        current_combined_score = calculate_combined_score_dbscan_normalized_v2(
            silhouette_avg=row['silhouette_avg'],
            negative_silhouette_ratio=row['negative_silhouette_ratio'],
            noise_ratio=row['noise_ratio'],
            num_clusters=row['unique_clusters'],
            min_overall_clusters=min_overall_clusters,
            max_overall_clusters=max_overall_clusters,
            target_min_clusters=target_min_clusters,
            target_max_clusters=target_max_clusters
        )
        
        # Add the combined score as a new column to the DataFrame
        df.at[index, 'combined_score'] = current_combined_score

        # Update best score and parameters if current score is better
        if current_combined_score > best_score:
            best_score = current_combined_score
            best_params = {
                'eps': row['eps'],
                'min_samples': row['min_samples']
            }
            best_metrics = {
                'silhouette_avg': row['silhouette_avg'],
                'negative_silhouette_ratio': row['negative_silhouette_ratio'],
                'noise_ratio': row['noise_ratio'],
                'unique_clusters': row['unique_clusters']
            }

    # --- Display Results ---
    print(f"\n--- DBSCAN Grid Search for {name} Completed ---")
    print(f"Best Parameters: {best_params}")
    print(f"Best Combined Score (Normalized): {best_score:.4f}")
    print(f"Corresponding Metrics (Raw Values):")
    for metric, value in best_metrics.items():
        print(f"  {metric}: {value:.4f}")
    
    return df, best_params, best_metrics, best_score

In [ ]:
dbscan_tsne_df = pd.read_pickle('data/processed_data/pkl/cluster_results/dbscan_grid_search_results_tsne.pickle')
dbscan_full_space_df = pd.read_pickle('data/processed_data/pkl/cluster_results/dbscan_grid_search_results_full_space.pickle')
# Option 1: t-SNE Daten
# Nehmen wir an, für t-SNE Daten wünschen wir 3-7 Cluster
tsne_df_evaluated, tsne_best_params, tsne_best_metrics, tsne_best_score = evaluate_dbscan_results(
    dbscan_tsne_df.copy(), # .copy() um sicherzustellen, dass der Original-DF nicht direkt modifiziert wird
    "t-SNE Data",
    target_min_clusters=2,
    target_max_clusters=25
)

# Option 2: Full Feature Space Daten
# Nehmen wir an, für Full Feature Space Daten wünschen wir 5-15 Cluster
full_space_df_evaluated, full_space_best_params, full_space_best_metrics, full_space_best_score = evaluate_dbscan_results(
    dbscan_full_space_df.copy(), # .copy() aus dem gleichen Grund
    "Full Feature Space Data",
    target_min_clusters=2,
    target_max_clusters=25
)

# Optional: Anzeigen der DataFrames mit den hinzugefügten Scores
print("\n--- Evaluated t-SNE DataFrame (first 5 rows) ---")
display(tsne_df_evaluated.head())

print("\n--- Evaluated Full Feature Space DataFrame (first 5 rows) ---")
display(full_space_df_evaluated.head())

## Vizualization

In [ ]:
# File path for the seed
SEED_FILE = "data/processed_data/pkl/random_seed.pkl"

# Function to save the seed
def save_random_seed(seed):
    with open(SEED_FILE, 'wb') as f:
        pickle.dump(seed, f)
    print(f"Random seed {seed} saved to {SEED_FILE}")

# Function to load the seed
def load_random_seed():
    if os.path.exists(SEED_FILE):
        with open(SEED_FILE, 'rb') as f:
            seed = pickle.load(f)
        print(f"Random seed {seed} loaded from {SEED_FILE}")
        return seed
    else:
        print(f"No seed file found at {SEED_FILE}. Generating a new seed.")
        return None

# Function to generate the color mapping
def generate_color_mapping_hsv(labels):
    unique_labels = np.unique(labels)
    cluster_labels = unique_labels[unique_labels != -1]
    num_clusters = len(cluster_labels)

    all_colors = []
    # Collect colors from 'tab10' colormap
    for i in range(plt.cm.get_cmap('tab10').N):
        all_colors.append(plt.cm.get_cmap('tab10')(i))
    # Collect colors from 'tab20' colormap
    for i in range(plt.cm.get_cmap('tab20').N):
        all_colors.append(plt.cm.get_cmap('tab20')(i))

    # Select enough colors for the clusters, up to the total available
    selected_colors = all_colors[:min(num_clusters, len(all_colors))]

    # --- IMPORTANT: Integrate seed logic here ---
    loaded_seed = load_random_seed()
    if loaded_seed is not None:
        random.seed(loaded_seed)
    else:
        # Generate a new seed and save it if none was found
        new_seed = random.randint(0, 2**32 - 1) # Random seed in the range 0 to 2^32 - 1
        random.seed(new_seed)
        save_random_seed(new_seed)
    # --- End of seed logic ---

    # Shuffle colors - this is now deterministic if the seed is loaded
    random.shuffle(selected_colors)

    custom_cmap = mcolors.ListedColormap(selected_colors)

    color_dict = {}
    for i, label in enumerate(cluster_labels):
        rgba_color = custom_cmap(i % len(selected_colors))
        # Convert RGBA to Hexadecimal color format
        hex_color = "#{:02x}{:02x}{:02x}".format(
            int(rgba_color[0] * 255),
            int(rgba_color[1] * 255),
            int(rgba_color[2] * 255)
        )
        color_dict[label] = hex_color

    # Assign black color to noise points (-1 label)
    if -1 in unique_labels:
        color_dict[-1] = "#000000"

    return color_dict

In [ ]:
full_data = df_scaled.iloc[:, 1:-4] 

# ---------------------------
# HDBSCAN – Full Feature Space
# ---------------------------

# Initialize and fit HDBSCAN model for the full feature space
hdbscan_full_model = hdbscan.HDBSCAN(
    min_cluster_size=34,
    min_samples=1
)
hdbscan_labels_full = hdbscan_full_model.fit_predict(full_data)

# Create a DataFrame for HDBSCAN results in full feature space
df_hdbscan_full = pd.DataFrame(full_data).copy()
df_hdbscan_full['cluster_id'] = hdbscan_labels_full
hdbscan_full_counts = Counter(hdbscan_labels_full) # Count occurrences of each cluster label

# Assign colors based on cluster IDs for HDBSCAN in full feature space
color_dict_hdbscan_full = generate_color_mapping_hsv(hdbscan_labels_full)
df_hdbscan_full['color'] = df_hdbscan_full['cluster_id'].map(color_dict_hdbscan_full)


# ---------------------------
# DBSCAN – Full Feature Space
# ---------------------------

# Initialize and fit DBSCAN model for the full feature space
dbscan_full_model = DBSCAN(
    eps=4.8,
    min_samples=11
)
dbscan_labels_full = dbscan_full_model.fit_predict(full_data)

# Create a DataFrame for DBSCAN results in full feature space
df_dbscan_full = pd.DataFrame(full_data).copy()
df_dbscan_full['cluster_id'] = dbscan_labels_full
dbscan_full_counts = Counter(dbscan_labels_full) # Count occurrences of each cluster label

# Assign colors based on cluster IDs for DBSCAN in full feature space
color_dict_dbscan_full = generate_color_mapping_hsv(dbscan_labels_full)
df_dbscan_full['color'] = df_dbscan_full['cluster_id'].map(color_dict_dbscan_full)


# ---------------------------
# DBSCAN – t-SNE Space
# ---------------------------

# Assuming X_tsne is already defined and loaded (e.g., from t-SNE reduction)

# Initialize and fit DBSCAN model for the t-SNE reduced space
dbscan_tsne_model = DBSCAN(
    eps=5.1,
    min_samples=42
)
dbscan_labels_tsne = dbscan_tsne_model.fit_predict(X_tsne)
dbscan_tsne_counts = Counter(dbscan_labels_tsne) # Count occurrences of each cluster label

# Create DataFrame for t-SNE results
# Note: X_tsne is a NumPy array, so assign column names
df_dbscan_tsne = pd.DataFrame(X_tsne, columns=['t-SNE Dim 1', 't-SNE Dim 2'])
df_dbscan_tsne['cluster_id'] = dbscan_labels_tsne

# Assign colors based on cluster IDs for DBSCAN in t-SNE space
color_dict_dbscan_tsne = generate_color_mapping_hsv(dbscan_labels_tsne)
df_dbscan_tsne['color'] = df_dbscan_tsne['cluster_id'].map(color_dict_dbscan_tsne)


# ---------------------------
# HDBSCAN – t-SNE Space
# ---------------------------

# Initialize and fit HDBSCAN model for the t-SNE reduced space
hdbscan_tsne_model = hdbscan.HDBSCAN(
    min_cluster_size=45,
    min_samples=3
)
hdbscan_labels_tsne = hdbscan_tsne_model.fit_predict(X_tsne)
hdbscan_tsne_counts = Counter(hdbscan_labels_tsne) # Count occurrences of each cluster label

# Create DataFrame for t-SNE results
df_hdbscan_tsne = pd.DataFrame(X_tsne, columns=['t-SNE Dim 1', 't-SNE Dim 2'])
df_hdbscan_tsne['cluster_id'] = hdbscan_labels_tsne

# Assign colors based on cluster IDs for HDBSCAN in t-SNE space
color_dict_hdbscan_tsne = generate_color_mapping_hsv(hdbscan_labels_tsne)
df_hdbscan_tsne['color'] = df_hdbscan_tsne['cluster_id'].map(color_dict_hdbscan_tsne)

print("\n--- Clustering completed. DataFrames with cluster IDs and colors created. ---")

In [ ]:
# Cluster plotting function with annotation and legend
def plot_clusters(ax, embedding, cluster_labels, counts, col_dict, method_label, xlim=None):
    """
    Plots cluster results on an axes object.

    Args:
        ax (matplotlib.axes.Axes): The axes object to plot on.
        embedding (np.array): The 2D embedding (e.g., t-SNE results).
        cluster_labels (np.array): The assigned cluster labels for each point.
        counts (collections.Counter): Counts of points per label.
        col_dict (dict): Dictionary for color mapping for each label.
        method_label (str): Label for the method used (top right of the plot).
        xlim (tuple, optional): Limits for the X-axis. Default is None.
    """
    for label in np.unique(cluster_labels):
        idx = (cluster_labels == label)
        
        # Set color and alpha value
        if label == -1:
            plot_color = 'blue'  # Noise points in blue
            plot_alpha = 1.0     # No alpha for noise points
            label_name = f'Noise ({counts[label]})' # Legend label for noise points
        else:
            plot_color = col_dict[label] # Cluster color from dictionary
            plot_alpha = 0.6             # Alpha for cluster points
            label_name = None # No legend label for clusters
            
        ax.scatter(embedding[idx, 0], embedding[idx, 1],
                   color=plot_color, label=label_name, alpha=plot_alpha, s=1)
    
    # Adjust X-axis limit if provided
    if xlim:
        ax.set_xlim(xlim)
    
    # Show noise legend only if noise is present
    if -1 in counts:
        ax.legend(loc='lower left', fontsize=8) # Smaller font for legend

    # Method label in the top right corner
    ax.text(0.95, 0.90, method_label, transform=ax.transAxes, fontsize=10,
            verticalalignment='bottom', horizontalalignment='right',
            bbox=dict(facecolor='white', edgecolor='none', boxstyle='round,pad=0.3'))

    # Circle annotation for cluster IDs
    x_range = ax.get_xlim()[1] - ax.get_xlim()[0]
    radius = 0.029 * x_range # Relative radius for the circles
    y_offset = radius * -1.5 # Y-offset for positioning the circle
    for label in np.unique(cluster_labels):
        if label == -1: # Do not annotate noise points
            continue
        idx = (cluster_labels == label)
        # Center of the cluster for annotation
        x_center = np.mean(embedding[idx, 0])
        y_center = np.mean(embedding[idx, 1])
        circle_center = (x_center, y_center + y_offset)
        circle = plt.Circle(circle_center, radius, color='white', ec='black', zorder=10)
        ax.add_patch(circle)
        ax.text(circle_center[0], circle_center[1], str(label),
                fontsize=10, color='black',
                horizontalalignment='center', verticalalignment='center', zorder=11)


# ---------------------------
# Create 2x2 Subplot Layout
# ---------------------------
fig, axes = plt.subplots(2, 2, figsize=(8, 8)) # Slightly larger figure for better visibility

# Plot 1 (Top Left): DBSCAN – Full Feature Space
# Labels were generated on the full feature space, but visualized here on the t-SNE embedding.
plot_clusters(axes[0, 0], X_tsne, dbscan_labels_full, dbscan_full_counts, color_dict_dbscan_full,
              'DBSCAN (full features)')

# Plot 2 (Top Right): DBSCAN – t-SNE Space
plot_clusters(axes[0, 1], X_tsne, dbscan_labels_tsne, dbscan_tsne_counts, color_dict_dbscan_tsne,
              'DBSCAN (t-SNE space)')

# Plot 3 (Bottom Left): HDBSCAN – Full Feature Space
# Labels were generated on the full feature space, but visualized here on the t-SNE embedding.
plot_clusters(axes[1, 0], X_tsne, hdbscan_labels_full, hdbscan_full_counts, color_dict_hdbscan_full,
              'HDBSCAN (full features)')

# Plot 4 (Bottom Right): HDBSCAN – t-SNE Space
plot_clusters(axes[1, 1], X_tsne, hdbscan_labels_tsne, hdbscan_tsne_counts, color_dict_hdbscan_tsne,
              'HDBSCAN (t-SNE space)')


# Remove axis labels for a cleaner look
for ax in axes.flat:
    ax.set_xticks([])
    ax.set_yticks([])

# Adjust spacing
plt.subplots_adjust(hspace=0.0, wspace=0.0)

# Save the figure
plt.savefig('cluster_comparison_plot.png', dpi=600, bbox_inches='tight')
# plt.savefig('cluster_comparison_plot.svg', format='svg', bbox_inches='tight') # Uncomment if SVG is desired

# Show the plot
plt.show()

In [ ]:
full_data = df_scaled.iloc[:, 1:] 


# HDBSCAN – Full Feature Space Model
hdbscan_full_model = hdbscan.HDBSCAN(
    min_cluster_size=34,
    min_samples=1
)
# Re-fit to ensure we have the model object with persistence attribute
hdbscan_labels_full = hdbscan_full_model.fit_predict(full_data)

# DBSCAN – Full Feature Space Model (no persistence needed for DBSCAN)
dbscan_full_model = DBSCAN(
    eps=4.8,
    min_samples=11
)
dbscan_labels_full = dbscan_full_model.fit_predict(full_data)


# HDBSCAN – t-SNE Space Model
hdbscan_tsne_model = hdbscan.HDBSCAN(
    min_cluster_size=45,
    min_samples=3
)
hdbscan_labels_tsne = hdbscan_tsne_model.fit_predict(X_tsne)

# DBSCAN – t-SNE Space Model
dbscan_tsne_model = DBSCAN(
    eps=5.1,
    min_samples=42
)
dbscan_labels_tsne = dbscan_tsne_model.fit_predict(X_tsne)


# --- Data for t-SNE Space ---
data_tsne_space = X_tsne

# --- Function to calculate clustering metrics for a clustering result ---
def calculate_clustering_metrics(data_embedding, labels, setup_name, model=None):
    """
    Calculates the defined clustering metrics for a given set of labels and data.

    Args:
        data_embedding (np.array or pd.DataFrame): The data (features) on which clustering is based.
        labels (np.array): The labels assigned by the clustering method.
        setup_name (str): A descriptive name for this clustering setup.
        model (object, optional): The trained clustering model (especially for HDBSCAN persistence).

    Returns:
        dict: A dictionary containing the calculated metrics.
    """
    results = {'Setup': setup_name}

    # 1. Number of Clusters (excluding noise)
    unique_clusters = len(set(labels) - {-1})
    results['Number of Clusters'] = unique_clusters

    # 2. Percentage of Noise
    noise_count = np.sum(labels == -1)
    total_points = len(labels)
    results['Noise Ratio (%)'] = (noise_count / total_points) * 100

    # Consider only points assigned to a cluster (not noise)
    clustered_points_indices = labels != -1
    
    # Check if enough points are available for silhouette and density-based metrics
    # (at least 2 clusters OR at least 2 points in clusters)
    if unique_clusters < 2 or np.sum(clustered_points_indices) < 2:
        results['Average Silhouette Score'] = np.nan
        results['Negative Silhouette Ratio (%)'] = np.nan
        results['Davies-Bouldin Index'] = np.nan
        results['Calinski-Harabasz Index'] = np.nan
    else:
        # Filter data and labels to clustered points
        filtered_data = data_embedding[clustered_points_indices]
        filtered_labels = labels[clustered_points_indices]

        # 3. Average Silhouette Score
        results['Average Silhouette Score'] = silhouette_score(filtered_data, filtered_labels)

        # 4. Percentage of Negative Silhouette Values
        silhouette_vals = silhouette_samples(filtered_data, filtered_labels)
        results['Negative Silhouette Ratio (%)'] = np.sum(silhouette_vals < 0) / len(silhouette_vals) * 100
        
        # 5. Davies-Bouldin Index
        results['Davies-Bouldin Index'] = davies_bouldin_score(filtered_data, filtered_labels)
        
        # 6. Calinski-Harabasz Index
        results['Calinski-Harabasz Index'] = calinski_harabasz_score(filtered_data, filtered_labels)
    
    # 7. HDBSCAN Persistence (if model available and HDBSCAN)
    if model and hasattr(model, 'cluster_persistence_') and model.cluster_persistence_ is not None and len(model.cluster_persistence_) > 0:
        results['Average HDBSCAN Persistence'] = np.mean(model.cluster_persistence_)
    else:
        results['Average HDBSCAN Persistence'] = np.nan # Not applicable or not available
    
    return results

# List to store results of all four scenarios
all_comparison_metrics = []

print("Calculating metrics for clustering setups...")

# --- Metrics for HDBSCAN – Full Feature Space ---
metrics_hdbscan_full_features = calculate_clustering_metrics(
    data_embedding=full_data,
    labels=hdbscan_labels_full,
    setup_name="HDBSCAN (Full Features)",
    model=hdbscan_full_model # Pass the model for persistence
)
all_comparison_metrics.append(metrics_hdbscan_full_features)

# --- Metrics for DBSCAN – Full Feature Space ---
metrics_dbscan_full_features = calculate_clustering_metrics(
    data_embedding=full_data,
    labels=dbscan_labels_full,
    setup_name="DBSCAN (Full Features)",
    model=None # No HDBSCAN model for persistence
)
all_comparison_metrics.append(metrics_dbscan_full_features)

# --- Metrics for HDBSCAN – t-SNE Space ---
metrics_hdbscan_tsne_space = calculate_clustering_metrics(
    data_embedding=data_tsne_space,
    labels=hdbscan_labels_tsne,
    setup_name="HDBSCAN (t-SNE Space)",
    model=hdbscan_tsne_model # Pass the model for persistence
)
all_comparison_metrics.append(metrics_hdbscan_tsne_space)

# --- Metrics for DBSCAN – t-SNE Space ---
metrics_dbscan_tsne_space = calculate_clustering_metrics(
    data_embedding=data_tsne_space,
    labels=dbscan_labels_tsne,
    setup_name="DBSCAN (t-SNE Space)",
    model=None # No HDBSCAN model for persistence
)
all_comparison_metrics.append(metrics_dbscan_tsne_space)


# --- Create the comparison table ---
comparison_df = pd.DataFrame(all_comparison_metrics)

# --- Column order for better readability ---
desired_columns = [
    'Setup',
    'Number of Clusters',
    'Noise Ratio (%)',
    'Average Silhouette Score',
    'Negative Silhouette Ratio (%)',
    'Davies-Bouldin Index',
    'Calinski-Harabasz Index',
    'Average HDBSCAN Persistence'
]
comparison_df = comparison_df[desired_columns]


# --- Formatting for better readability ---
# For Silhouette Score, Davies-Bouldin, Calinski-Harabasz, and Persistence
for col in ['Average Silhouette Score', 'Davies-Bouldin Index', 'Calinski-Harabasz Index', 'Average HDBSCAN Persistence']:
    comparison_df[col] = comparison_df[col].map(
        lambda x: f'{x:.4f}' if pd.notna(x) else 'N/A'
    )
# For percentage values
for col in ['Noise Ratio (%)', 'Negative Silhouette Ratio (%)']:
    comparison_df[col] = comparison_df[col].map(
        lambda x: f'{x:.2f}%' if pd.notna(x) else 'N/A'
    )

# --- Display the table ---
print("\n--- Quantitative Comparison of Clustering Results ---")
print(comparison_df.to_string(index=False))


## Pairwise Metrics

In [ ]:
# -------------------------------
# Optional: Pairwise comparison of clusterings (ARI and NMI)
# -------------------------------
# These are external metrics comparing label similarity (even without ground truth)

pairwise_metrics = pd.DataFrame({
    'HDBSCAN (full feature set) vs HDBSCAN (t-SNE)': [
        metrics.adjusted_rand_score(hdbscan_labels_full, hdbscan_labels_tsne),
        metrics.normalized_mutual_info_score(hdbscan_labels_full, hdbscan_labels_tsne)
    ],
    'HDBSCAN (full feature set) vs DBSCAN (t-SNE)': [
        metrics.adjusted_rand_score(hdbscan_labels_full, dbscan_labels_tsne),
        metrics.normalized_mutual_info_score(hdbscan_labels_full, dbscan_labels_tsne)
    ],
    'HDBSCAN (t-SNE) vs DBSCAN (t-SNE)': [
        metrics.adjusted_rand_score(hdbscan_labels_tsne, dbscan_labels_tsne),
        metrics.normalized_mutual_info_score(hdbscan_labels_tsne, dbscan_labels_tsne)
    ]
}, index=['ARI', 'NMI'])

print("\nPairwise comparison metrics between methods:")
display(pairwise_metrics.T)


# Vizualizations

## t-SNE with heatmap for every single feature

In [ ]:

# Assuming df_original_features, df_scaled_features, and X_tsne are already defined.

# Create the feature ID dictionary
feature_id_dict = {feature: "F" + str(i + 1) for i, feature in enumerate(df_original_features.columns)}
df_vis = df_scaled_features.copy()

# Iterate over the columns to create scatter plots
for col in df_scaled_features:
    plt.figure(figsize=(8, 5))
    scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1],
                          c=df_scaled_features[col],
                          cmap='viridis',
                          alpha=0.5,
                          s=5)

    # Add color bar legend
    cbar = plt.colorbar(scatter, label=col)
    label_text = feature_id_dict[col]

    # Place the label as a textbox in the bottom right
    plt.text(0.95, 0.05, label_text,
             transform=plt.gca().transAxes,
             horizontalalignment='right',
             verticalalignment='bottom',
             fontsize=20,
             bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))

    plt.xlabel('t-SNE 1')
    plt.ylabel('t-SNE 2')

    plt.tight_layout()
    plt.show()

In [ ]:
# Assuming X_tsne, df_scaled_features, and feature_id_dict are already defined.

fig, axs = plt.subplots(2, 2, figsize=(8, 8))
axs = axs.flatten()  # Allows easy access by index

features_to_plot = ['total_emp_occu_51-4033', 'total_emp_naics_3335', 'total_emp_naics_3364', 'total_emp_naics_3366']

# Common scaling for all four plots
all_data_common = np.concatenate([
    df_scaled_features[col].values for col in features_to_plot
])
vmin_common, vmax_common = all_data_common.min(), all_data_common.max()

for i, col in enumerate(features_to_plot):
    ax = axs[i]

    data = df_scaled_features[col]
    label_text = feature_id_dict[col]

    scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1],
                         c=data,
                         cmap='viridis',
                         alpha=0.5,
                         s=2,
                         vmin=vmin_common,
                         vmax=vmax_common)

    # Place the textbox with the label in the bottom right of the plot
    ax.text(0.95, 0.05, label_text,
            transform=ax.transAxes,
            horizontalalignment='right',
            verticalalignment='bottom',
            fontsize=15,
            bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))

    # Remove axes and axis labels
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

# Common colorbar for all four plots
cbar_ax = fig.add_axes([0.15, 0.05, 0.7, 0.02])  # [left, bottom, width, height]
cbar = fig.colorbar(axs[0].collections[0], cax=cbar_ax, orientation='horizontal')

plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.subplots_adjust(hspace=0.0, wspace=0.0)  # Prevents overlap with the colorbar
plt.savefig('tsne_feature_values.png', dpi=600)
plt.show()

## Cluster on map

In [ ]:
df_scaled['cluster_id'] = df_hdbscan_tsne['cluster_id']
df_scaled['color'] = df_hdbscan_tsne['color']

numeric_columns = df_scaled.columns[1:-3].tolist()
# Calculation of means for numeric columns per cluster
mean_values = df_scaled.groupby('cluster_id')[numeric_columns].mean()

# Calculation of overall_mean as the average across all numeric columns
mean_values['overall_mean'] = mean_values.mean(axis=1)

# Sorting by overall_mean in descending order
mean_values = mean_values.sort_values(by='overall_mean', ascending=False)

# Adding a 'rank' column based on overall_mean
mean_values['rank'] = mean_values['overall_mean'].rank(method='dense', ascending=False).astype(int)
mean_values = mean_values.reset_index()
mean_values.head()
df_scaled = df_scaled.merge(mean_values[['rank', 'cluster_id']], on='cluster_id', how='left')
df_scaled['overall_mean'] = df_scaled.iloc[:,1:-3].mean(axis=1)
df_scaled.head()

### one map 

In [ ]:
# Filter Counties and States for Alaska (STATEFP=02)
filtered_county_shape_north_america_merged = filtered_county_shape_north_america.merge(df_scaled[['FIPS', 'cluster_id','color','rank','overall_mean']], on='FIPS', how='left')
filtered_county_shape_alaska_merged = filtered_county_shape_alaska.merge(df_scaled[['FIPS', 'cluster_id','color','rank','overall_mean']], on='FIPS', how='left')

alaska_state = filtered_state_shape_north_america[filtered_state_shape_north_america['STATEFP'] == '02']
alaska_counties = filtered_county_shape_north_america_merged[filtered_county_shape_north_america_merged['STATEFP'] == '02']

# Filter remaining data
remaining_states = filtered_state_shape_north_america[filtered_state_shape_north_america['STATEFP'] != '02']
remaining_counties = filtered_county_shape_north_america_merged[filtered_county_shape_north_america_merged['STATEFP'] != '02']

# Create the plot
fig, ax = plt.subplots(figsize=(15, 7))

# Plot remaining counties and states
remaining_counties.plot(
    facecolor=remaining_counties['color'],
    linewidth=0.4,
    ax=ax,
    edgecolor='grey',
    legend=False
)
remaining_states.plot(ax=ax, color='none', edgecolor='black', linewidth=0.5)

# Add a separate inset for Alaska
ak_ax = fig.add_axes([0.62, 0.255, 0.275, 0.275]) # Position and size of the Alaska inset

# Optionally clip Alaska to a polygon if needed
polygon = Polygon([(-180, 50), (-180, 72), (-140, 72), (-140, 50)]) # Define bounding polygon
alaska_state = alaska_state.clip(polygon) # Clip Alaska state shapes
alaska_counties = alaska_counties.clip(polygon) # Clip Alaska counties

# Plot Alaska in the inset
alaska_counties.plot(
    facecolor=alaska_counties['color'],
    linewidth=0.4,
    ax=ak_ax,
    edgecolor='grey',
    legend=False
)
alaska_state.plot(ax=ak_ax, color='none', edgecolor='black', linewidth=0.8)

# Customize Alaska inset
ak_ax.axis('off') # Turn off the axes
ax.axis('off') # Turn off the axes

# Add the legend for clusters to the main plot
unique_clusters = sorted(filtered_county_shape_north_america_merged['cluster_id'].unique())
cluster_counts = filtered_county_shape_north_america_merged['cluster_id'].value_counts()
legend_elements = []

for cluster in unique_clusters:
    if cluster == -1:
        count = cluster_counts.get(-1, 0)
        legend_elements.append(Patch(facecolor='blue', label=f'noise ({count})'))
    else:
        color = filtered_county_shape_north_america_merged.loc[filtered_county_shape_north_america_merged['cluster_id'] == cluster, 'color'].values[0]
        count = cluster_counts.get(cluster, 0)
        legend_elements.append(Patch(facecolor=color, label=f'{cluster} ({count})'))

ax.legend(handles=legend_elements, title='Cluster ID', loc='lower left',
    title_fontsize=10, # Set the font size of the legend title
    fontsize=8 )

plt.axis('off')
plt.show()

In [ ]:
# Merge cluster data with county shapes
filtered_county_shape_north_america_merged = filtered_county_shape_north_america.merge(df_scaled[['FIPS', 'cluster_id','color','rank','overall_mean']], on='FIPS', how='left')
filtered_county_shape_alaska_merged = filtered_county_shape_alaska.merge(df_scaled[['FIPS', 'cluster_id','color','rank','overall_mean']], on='FIPS', how='left')


filtered_counties = filtered_county_shape_north_america_merged[
    (filtered_county_shape_north_america_merged['rank'] <= 5) |
    (filtered_county_shape_north_america_merged['cluster_id'] == -1)
]
top_rank = filtered_counties[filtered_counties['rank'] <= 5]

# Filter counties from cluster -1, sort them by overall_mean in descending order, and select the top 20
cluster_minus_one_top20 = (
    filtered_counties[filtered_counties['cluster_id'] == -1]
    .sort_values(by='overall_mean', ascending=False)
    .head(20)
)

# Combine both sub-dataframes and remove any duplicates
filtered_counties = pd.concat([top_rank, cluster_minus_one_top20]).drop_duplicates()


# Original data (for outlines/contours)
all_counties = filtered_county_shape_north_america_merged.copy()

# Split into Alaska and Rest
# For Alaska:
alaska_state = filtered_state_shape_north_america[
    filtered_state_shape_north_america['STATEFP'] == '02'
]
alaska_counties_fill = filtered_counties[
    filtered_counties['STATEFP'] == '02'
]
alaska_counties_all = all_counties[
    all_counties['STATEFP'] == '02'
]

# For remaining states:
remaining_states = filtered_state_shape_north_america[
    filtered_state_shape_north_america['STATEFP'] != '02'
]
remaining_counties_fill = filtered_counties[
    filtered_counties['STATEFP'] != '02'
]
remaining_counties_all = all_counties[
    all_counties['STATEFP'] != '02'
]

# 2. Create the main plot
fig, ax = plt.subplots(figsize=(15, 7))

# Plot non-noise counties with their assigned colors
remaining_counties_non_noise = remaining_counties_fill[remaining_counties_fill['cluster_id'] != -1]
if not remaining_counties_non_noise.empty:
    remaining_counties_non_noise.plot(
        facecolor=remaining_counties_non_noise['color'],
        linewidth=0.2,
        ax=ax,
        edgecolor='none',
        legend=False
    )

# Plot noise counties explicitly in blue
remaining_counties_noise = remaining_counties_fill[remaining_counties_fill['cluster_id'] == -1]
if not remaining_counties_noise.empty:
    remaining_counties_noise.plot(
        facecolor='blue', # Explicitly set to blue for noise
        linewidth=0.2,
        ax=ax,
        edgecolor='none',
        legend=False
    )

# Additionally plot all outlines of the remaining counties (from the entire dataset)
remaining_counties_all.plot(
    facecolor='none',
    linewidth=0.2,
    ax=ax,
    edgecolor='grey',
    legend=False
)

# Plot the state contours
remaining_states.plot(
    ax=ax,
    color='none',
    edgecolor='black',
    linewidth=0.5
)

# 3. Plot Alaska in an inset
# Define a bounding polygon for clipping
polygon = Polygon([(-180, 50), (-180, 72), (-140, 72), (-140, 50)])
alaska_state_clipped = alaska_state.clip(polygon)

# Create the Alaska inset
ak_ax = fig.add_axes([0.62, 0.255, 0.275, 0.275])

# Plot non-noise Alaska counties with their assigned colors
alaska_counties_fill_non_noise = alaska_counties_fill[alaska_counties_fill['cluster_id'] != -1]
if not alaska_counties_fill_non_noise.empty:
    alaska_counties_fill_non_noise = alaska_counties_fill_non_noise.clip(polygon)
    alaska_counties_fill_non_noise.plot(
        facecolor=alaska_counties_fill_non_noise['color'],
        linewidth=0.2,
        ax=ak_ax,
        edgecolor='none',
        legend=False
    )

# Plot noise Alaska counties explicitly in blue
alaska_counties_fill_noise = alaska_counties_fill[alaska_counties_fill['cluster_id'] == -1]
if not alaska_counties_fill_noise.empty:
    alaska_counties_fill_noise = alaska_counties_fill_noise.clip(polygon)
    alaska_counties_fill_noise.plot(
        facecolor='blue', # Explicitly set to blue for noise
        linewidth=0.2,
        ax=ak_ax,
        edgecolor='none',
        legend=False
    )

# Plot all Alaska outlines (from the original dataset)
if not alaska_counties_all.empty:
    alaska_counties_all = alaska_counties_all.clip(polygon)
    alaska_counties_all.plot(
        facecolor='none',
        linewidth=0.2,
        ax=ak_ax,
        edgecolor='grey',
        legend=False
    )

# Plot the state contour of Alaska
alaska_state_clipped.plot(
    ax=ak_ax,
    color='none',
    edgecolor='black',
    linewidth=0.5
)

ak_ax.axis('off') # Hide axes in the Alaska inset
ax.axis('off')    # Hide main axes

unique_clusters = filtered_counties.groupby('cluster_id')['rank'].min().sort_values().index.tolist()
cluster_counts = filtered_counties['cluster_id'].value_counts()
legend_elements = []

for cluster in unique_clusters:
    if cluster == -1:
        # Handle the "noise" cluster separately
        count = cluster_counts.get(-1, 0)
        legend_elements.append(Patch(facecolor='blue', label=f'C-1, ({count})'))
    else:
        # Determine the minimum rank for each cluster
        min_rank = filtered_counties.loc[filtered_counties['cluster_id'] == cluster, 'rank'].min()
        color = filtered_counties.loc[filtered_counties['cluster_id'] == cluster, 'color'].values[0]
        count = cluster_counts.get(cluster, 0)
        legend_elements.append(Patch(facecolor=color, label=f'C{cluster}, ({count})'))

ax.legend(handles=legend_elements, loc='lower left', bbox_to_anchor=(0.06, 0.095),
          title_fontsize=12, fontsize=11, ncol=2)
plt.savefig('county_cluster_map.png', dpi=600)
plt.show()

## feature importance

The code trains a separate Random Forest model for each unique cluster to assess the importance of various features in distinguishing each cluster.  
The results are visualized in a heatmap, displaying the significance of each feature across different clusters.


In [ ]:

# Set the style for visualization
sns.set_context("paper", font_scale=1.2)
sns.set_style("whitegrid")

# Remove 'cluster_id', 'color', 'FIPS' etc.
X = df_scaled[df_scaled.columns[1:-4]]
clusters = sorted(df_scaled['cluster_id'].unique())

# Calculate feature importances and scale them to integers
feature_importances = [
    {"Cluster": cluster, "Feature": feature, "Importance": int(importance * 100)}
    for cluster in clusters
    for feature, importance in zip(X.columns, RandomForestClassifier(random_state=42).fit(X, (df_scaled['cluster_id'] == cluster).astype(int)).feature_importances_)
]

# Create DataFrame
importance_df = pd.DataFrame(feature_importances)

# Pivot table for the heatmap
pivot = importance_df.pivot(index="Feature", columns="Cluster", values="Importance")

# Sort clusters by 'rank'
cluster_order = df_scaled.groupby('cluster_id')['rank'].mean().sort_values().index
pivot = pivot[cluster_order]

# Transform feature names (F1, F2, ...)
feature_id_dict = {feature: f"F{i+1}" for i, feature in enumerate(df_original_features.columns)}
pivot.index = pivot.index.map(lambda x: feature_id_dict.get(x, x))

# Sort features in reverse numerical order
pivot = pivot.sort_index(key=lambda x: x.str.extract(r'(\d+)').astype(int)[0], ascending=False)

# Draw the heatmap
fig, ax = plt.subplots(figsize=(10, 8), dpi=600)
sns.heatmap(pivot, cmap="YlGnBu", annot=True, fmt="d",  # fmt="d" for integers

            annot_kws={"size": 10}, ax=ax)
ax.set_xlabel('Cluster ID', fontsize=12)
ax.set_ylabel('Features', fontsize=12)
plt.xticks(rotation=0, fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.tight_layout()

# Save and display the heatmap
plt.savefig("feature_importance_heatmap.png", format="png", dpi=600)
plt.show()

## Barplots with std

In [ ]:
# Assign 'cluster_id' and 'color' from df_scaled to df_original
df_original['cluster_id'] = df_scaled['cluster_id']
df_original['color'] = df_scaled['color']

# Global settings for compact display
plt.rcParams.update({
    'font.size': 10,
    'axes.titlesize': 10,
    'axes.labelsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10
})

# Mapping for NAICS codes
naics_plan_b = {
    "3315": "Foundries",
    "Automotive": "Automotive",
    "3364": "Aerospace",
    "3366": "Shipbuilding",
    "3335": "Metalworking Machines Manufacturing",
    "3320A1": "Steel forming",
    "3320A2": "Structural Metals Manufacturing",
    "3327": "Machine Shops",
    "3312": "Steel Product Manufacturing",
    "3314": "Nonferrous Metal Production"
}

# Mapping for Occupation codes
occupations_plan_b = {
    "51-9022": "Grinding, Polishing by Hand",
    "51-4121": "Welders, Cutters, Solderers",
    "49-9041": "Industrial Machinery Mechanics",
    "49-9071": "Maintenance and Repair Workers",
    "51-4033": "Grinding, Lapping, Polishing",
    "51-4035": "Milling and Planing Machine Setters",
    "47-2211": "Sheet Metal Workers",
    "51-2041": "Structural Metal Fabricators"
}

# Lists with the desired codes
naics_plan_b_keys = list(naics_plan_b.keys())
occupations_plan_b_keys = list(occupations_plan_b.keys())

# Cluster selection
selected_clusters = [11,10,9,8,7]

# Remove first and last column
df = df_original.iloc[:, 1:-1]

# Identify columns
naics_cols = [col for col in df.columns if "naics" in col]
occ_cols = [col for col in df.columns if "occ" in col]

# Extract codes
def extract_code(col_name):
    match = re.search(r'(\d{4,}A?\d*|Automotive|\d{2}-\d{4})$', col_name)
    return match.group() if match else None

# Mapping: Original column names → Extracted codes
occ_mapping = {col: extract_code(col) for col in occ_cols}
naics_mapping = {col: extract_code(col) for col in naics_cols}

# Keep only the desired codes from the lists
occ_cols_filtered = [col for col, code in occ_mapping.items() if code in occupations_plan_b_keys]
naics_cols_filtered = [col for col, code in naics_mapping.items() if code in naics_plan_b_keys]

# Adjust labels accordingly
occ_labels = [occupations_plan_b[occ_mapping[col]] for col in occ_cols_filtered]
naics_labels = [naics_plan_b[naics_mapping[col]] for col in naics_cols_filtered]

# Function to wrap label text a maximum of once (i.e., at most two lines)
def wrap_label(label, width=15):
    wrapped = textwrap.wrap(label, width=width)
    if len(wrapped) > 1:
        return wrapped[0] + "\n" + wrapped[1]
    else:
        return wrapped[0]

def wrap_label(label, width=19):
    wrapped = textwrap.wrap(label, width=width)
    return "\n".join(wrapped[:3])  # maximum three lines

wrapped_occ_labels = [wrap_label(label) for label in occ_labels]
wrapped_naics_labels = [wrap_label(label) for label in naics_labels]

# Calculate cluster means and standard deviations
cluster_means = df.groupby("cluster_id").mean()
cluster_stds = df.groupby("cluster_id").std()

# Create a dictionary that assigns a color to each cluster
cluster_colors = {}
for cluster in selected_clusters:
    color_val = df_original.loc[df_original["cluster_id"] == cluster, "color"].iloc[0]
    cluster_colors[cluster] = color_val

# Compact parameters
bar_width = 0.2    # narrower bars
n_clusters = len(selected_clusters)
group_gap = 0.3      # smaller spacing between feature groups

# Calculate y-positions for OCC Features
indices_occ = np.arange(len(wrapped_occ_labels)) * (n_clusters * bar_width + group_gap)
y_ticks_occ = indices_occ + (n_clusters * bar_width) / 2

# Calculate y-positions for NAICS Features
indices_naics = np.arange(len(wrapped_naics_labels)) * (n_clusters * bar_width + group_gap)
y_ticks_naics = indices_naics + (n_clusters * bar_width) / 2

# Create the plot with a smaller overall size
fig, axes = plt.subplots(1, 2, figsize=(8, 6))

# Error bar settings: thinner lines for the whiskers
error_kw = {'elinewidth': 0.5, 'capsize': 3}

# OCC Features Plot
if occ_cols_filtered:
    for i, cluster in enumerate(selected_clusters):
        means = cluster_means.loc[cluster, occ_cols_filtered]
        stds = cluster_stds.loc[cluster, occ_cols_filtered]
        positions = indices_occ + i * bar_width
        axes[0].barh(positions,
                     means.values,
                     height=bar_width,
                     xerr=stds.values,
                     label=f"Cl. {cluster}",
                     error_kw=error_kw,
                     color=cluster_colors[cluster])
    axes[0].set_yticks(y_ticks_occ)
    axes[0].set_yticklabels(wrapped_occ_labels, ha='left')
    axes[0].set_title("Occupation features")
    axes[0].set_xlabel("mean value")
    axes[0].set_xlim(0, 3000)
    axes[0].tick_params(axis='y', pad=105)
    axes[0].legend(loc='upper right', frameon=False)

# NAICS Features Plot
if naics_cols_filtered:
    for i, cluster in enumerate(selected_clusters):
        means = cluster_means.loc[cluster, naics_cols_filtered]
        stds = cluster_stds.loc[cluster, naics_cols_filtered]
        positions = indices_naics + i * bar_width
        axes[1].barh(positions,
                     means.values,
                     height=bar_width,
                     xerr=stds.values,
                     label=f"Cl. {cluster}",
                     error_kw=error_kw,
                     color=cluster_colors[cluster])
    axes[1].set_yticks(y_ticks_naics)
    axes[1].set_yticklabels(wrapped_naics_labels, ha='left')
    axes[1].set_title("Industry features")
    axes[1].set_xlabel("mean value")
    axes[1].set_xlim(0, 2000)

    axes[1].tick_params(axis='y', pad=90)
    #axes[1].legend(loc='lower right', frameon=False) # This line is commented out in the original code, so keeping it commented.
plt.tight_layout(w_pad=0.0)

plt.savefig('cluster_feature_values.png', dpi=600)
plt.show()